# Proyecto EDA - Analisis Precios Supermercados - Esteban Mangado

><font size="4"> *“En los días que corren la gente sabe el precio de todo y el valor de nada.”* </font> - Oscar Wilde

# Índice

1. [Introducción](#1.-Introducción)  
2. [Obtención de los datos](#2.-Obtención-de-los-datos)  
3. [Librerias a utilizar](#3.-Librerias-a-utilizar)
4. [Importar datos de los CSV](#4.-Importar-datos-de-los-CSV)  
5. [Análisis de los datos](#5.-Análisis-de-los-datos)   
    5.1. [Análisis del DataFrame de Carrefour](#5.1.-Análisis-del-DataFrame-de-Carrefour)  
    5.2. [Análisis del DataFrame de Día](#5.2.-Análisis-del-DataFrame-de-Día)   
    5.3. [Análisis del DataFrame de Mercadona](#5.3.-Análisis-del-DataFrame-de-Mercadona)   
    5.4. [Primeras conclusiones](#5.4.-Primeras-conclusiones)       
6. [Limpieza de los datos](#6.-Limpieza-de-los-datos)   
    6.1. [Limpieza del DataFrame de Carrefour](#6.1.-Limpieza-del-DataFrame-de-Carrefour)  
    6.2. [Limpieza del DataFrame de Día](#6.2.-Limpieza-del-DataFrame-de-Día)   
    6.3. [Limpieza del DataFrame de Mercadona](#6.3.-Limpieza-del-DataFrame-de-Mercadona)     
7. [Exportación de los datos](#7.-Exportación-de-los-datos)   
    7.1. [Preparación de los datos](#7.1.-Preparación-de-los-datos)  
    7.2. [Conexión con la base de datos](#7.2.-Conexión-con-la-base-de-datos)  
    7.3. [Actualización de los datos](#7.3.-Actualización-de-los-datos)

# 1. Introducción
Este proyecto surge como idea de un comparador de precios entre los distintos supermercados.
Durante este último año han salido muchas noticias derivadas del aumento de los precios en todos los sectores, no solo en el de la alimentación.

![EDA%201.jpg](./img/EDA%201.jpg)

![EDA%202.jpg](./img/EDA%202.jpg)

![EDA%204.jpg](./img/EDA%204.jpg)

![EDA%203.jpg](./img/EDA%203.jpg)


Además, gracias a los datos obtenidos, se va a realizar una comparativa entre los precios de Mercadona, Día y Carrefour, estudiando su evolución temporal desde marzo de 2021 hasta el momento de la entrega de este proyecto.



# 2. Obtención de los datos
Los datos se han obtenido gracias a la iniciativa *Open Data de [DataMarket](https://datamarket.es/#productos-de-supermercados-dataset)* para dar acceso a datos actualizados y de calidad de temáticas que son de interés para la comunidad.

Esta iniciativa proporciona datos de tres de los supermercados más importantes del país (Mercadona, Día y Carrefour), con un volumen de datos diarios de aproximadamente unos 50000 registros, que van actualizando en unos CSV descargables desde un [enlace de drive](https://drive.google.com/drive/folders/1nQBn2mkZQyP-pAwD1-gBRVSbGh0unAvr). La serie histórica de datos comienza desde marzo de 2021, lo que permite realizar una comparativa del aumento de los precios en este último año.


No está programado para que descargue los datos desde ese mes, por lo que ha que hacerlo manualmente (no he encontrado la forma óptima de que Python descargue los archivos de Drive eligiendo qué archivos, ya que trata todo lo que esté dentro de la carpeta como un bloque y descargaría todo)

# 3. Librerías a utilizar

Aquí se declaran todas las librerías que van a ir utilizandose durante todo el proyecto

In [1]:
import pandas as pd
import glob
import numpy as np
from datetime import datetime
from openpyxl import Workbook, load_workbook
import sqlite3

In [2]:
now = datetime.now()

inicio = now.strftime("%H:%M:%S")
print("Hora de inicio =", inicio)

Hora de inicio = 18:48:24


# 4. Importar datos de los CSV
Los datos se proporcionan en distintos CSV, separados para cada supermercado y por mes.

Para importar estos datos, se ha realizado primero una importación completa de todos los CSV en tres variables (tablas de pandas), una para cada supermercado. Después estas tres tablas se limpian y se preparan para introducirlas en una base de datos Sqlite3 para representarlas en Power BI


**IMPORTANTE**  
Tanto la lectura de los csv como de los excel de las categorias que hay más adelante, se encuentran referidos con la ruta en la que se encuentran por lo que habrá que modificar la ruta para que funcione todo correctamente.

In [3]:
#Primero se indica la ruta donde se encuentran los CSV
path = r'C:\Users\empca\Documents\14 Bootcamp The Bridge\05 Proyecto EDA\Datos RAW\partition_by_supermarket_month'

#Se crean las variables temporales que leen los distintos CSV, uno para supermercado, que se utilizan en un bucle for
archivo_dia = glob.glob(path + "/*dia*.csv")
archivo_carrefour = glob.glob(path + "/*carrefour*.csv")
archivo_mercadona = glob.glob(path + "/*mercadona*.csv")

#Se crea una lista vacía donde se va a ir introduciendo cada csv leído, cada uno de estos en una posición de la lista
carrefour_raw = []
dia_raw = []
mercadorna_raw = []

#Bucles for para leer cada CSV e añadiéndolos a la lista vacía declarada antes
for file_d in archivo_dia:
    dia_raw.append(pd.read_csv(file_d))
    
for file_c in archivo_carrefour:
    carrefour_raw.append(pd.read_csv(file_c))
    
for file_m in archivo_mercadona:
    mercadorna_raw.append(pd.read_csv(file_m))
    

# Se concatenan todos los csv leidos e introducidos en las listas en un único DataFrame para cada supermercado
df_carrefour = pd.concat(carrefour_raw, ignore_index=True)
df_dia = pd.concat(dia_raw, ignore_index=True)
df_mercadona = pd.concat(mercadorna_raw, ignore_index=True)

Se muestra un pequeño fragmento de estos 3 dataframes con los que se va a empezar a trabajar

In [4]:
#Se muestra el tamaño de los distintos DataFrame
print("El DataFrame de Carrefour tiene",df_carrefour.shape[0], "de filas y",df_carrefour.shape[1], "de columnas")
print("El DataFrame de Dia tiene",df_dia.shape[0], "de filas y",df_dia.shape[1], "de columnas")
print("El DataFrame de Mercadona tiene",df_mercadona.shape[0], "de filas y",df_mercadona.shape[1], "de columnas")

El DataFrame de Carrefour tiene 1967509 de filas y 10 de columnas
El DataFrame de Dia tiene 3723032 de filas y 10 de columnas
El DataFrame de Mercadona tiene 3855618 de filas y 10 de columnas


In [5]:
#Se muestran las primeras filas del df de Carrefour
df_carrefour.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://www.carrefour.es/supermercado/hamburgu...,carrefour-es,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e
1,https://www.carrefour.es/supermercado/hamburgu...,carrefour-es,el_mercado_carniceria_hamburguesas,Hamburguesa de pollo sin gluten y sin lactosa ...,NaN,3.00,8.33,kg,2021-03-06,b02fc529217ae756011dad17df381298
2,https://www.carrefour.es/supermercado/hamburgu...,carrefour-es,el_mercado_carniceria_hamburguesas,Hamburguesa vegetal de champiñones Suquipá 180 g,NaN,2.25,12.50,kg,2021-03-06,d488a137436e5d8e70b9a5a3cd572fe0


In [6]:
df_dia.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://www.dia.es/compra-online/productos/dro...,dia-es,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5
1,https://www.dia.es/compra-online/productos/dro...,dia-es,drogueria_y_limpieza_cuidado_ropa__detergente_...,PUNTOMATIC detergente máquina blanco puro en p...,NaN,0.99,0.25,lavado,2021-03-06,e70c1708ea10f95a47e3ff9959a7e1fd
2,https://www.dia.es/compra-online/productos/dro...,dia-es,drogueria_y_limpieza_cuidado_ropa__detergente_...,WIPP EXPRESS detergente máquina polvo maleta 3...,NaN,8.99,0.26,lavado,2021-03-06,23aed350daad23fe99f1d7dad9c5e0b3


In [7]:
df_mercadona.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://tienda.mercadona.es/product/79439/cola...,mercadona-es,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-06,6dfd858ef08db9863e83c462f25bd5df
1,https://tienda.mercadona.es/product/79052/caps...,mercadona-es,fitoterapia_y_parafarmacia_fitoterapia,Cápsulas kaptogras Deliplus,Caja,4.5,27.44,100g,2021-03-06,9def5af6e14252efe09c744a5248c575
2,https://tienda.mercadona.es/product/79621/comp...,mercadona-es,fitoterapia_y_parafarmacia_fitoterapia,Comprimidos vitaminas y minerales Deliplus,Caja,3.0,10.15,100g,2021-03-06,6ff9a5d970dfb2383f0742a2314fa972


# 5. Análisis de los datos
Antes de empezar a trabajar con la limpieza de los datos, es necesario hacer un primer análisis de los datos de partida.

En este caso, aunque se trabaja con DataFrames de millones de filas, el tener únicamente 10 columnas facilita el trabajo.

# 5.1. Análisis del DataFrame de Carrefour

In [8]:
#Análisis general del DataFrame
df_carrefour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967509 entries, 0 to 1967508
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   url              object 
 1   supermarket      object 
 2   category         object 
 3   name             object 
 4   description      float64
 5   price            float64
 6   reference_price  float64
 7   reference_unit   object 
 8   insert_date      object 
 9   product_id       object 
dtypes: float64(3), object(7)
memory usage: 150.1+ MB


In [9]:
#Para ver la cantidad de nulos
df_carrefour.isnull().sum()

url                      0
supermarket              0
category                 0
name                   253
description        1967509
price                   12
reference_price          0
reference_unit           0
insert_date              0
product_id               0
dtype: int64

In [10]:
#Analisis de cuántos produtos únicos ha tenido Carrefour desde el comienzo del registro de datos
df_carrefour_registros_inicial = df_carrefour.shape[0]
df_carrefour_productos_unicos_inicial = len(df_carrefour.product_id.unique())
df_carrefour_productos_unicos_inicial

15441

In [11]:
round(df_carrefour.describe(),2)

,description,price,reference_price
count,0.0,1967497.00,1967509.00
mean,NaN,5.09,22.40
std,NaN,9.31,608.89
min,NaN,0.14,0.01
25%,NaN,1.60,1.33
50%,NaN,2.81,3.85
75%,NaN,5.40,9.16
max,NaN,595.00,140000.00


# 5.2. Análisis del DataFrame de Día

In [12]:
#Análisis general del DataFrame
df_dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723032 entries, 0 to 3723031
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   url              object 
 1   supermarket      object 
 2   category         object 
 3   name             object 
 4   description      float64
 5   price            float64
 6   reference_price  float64
 7   reference_unit   object 
 8   insert_date      object 
 9   product_id       object 
dtypes: float64(3), object(7)
memory usage: 284.0+ MB


In [13]:
#Para ver la cantidad de nulos
df_dia.isnull().sum()

url                      0
supermarket              0
category                 0
name                     0
description        3723032
price                    6
reference_price          6
reference_unit        5480
insert_date              0
product_id               0
dtype: int64

In [14]:
#Analisis de cuántos produtos únicos ha tenido Día desde el comienzo del registro de datos
df_dia_registros_inicial = df_dia.shape[0]
df_dia_productos_unicos_inicial = len(df_dia.product_id.unique())
df_dia_productos_unicos_inicial

10641

In [15]:
round(df_dia.describe(),2)

,description,price,reference_price
count,0.0,3723026.00,3723026.00
mean,NaN,3.41,28.93
std,NaN,5.27,830.16
min,NaN,0.08,0.02
25%,NaN,1.49,2.38
50%,NaN,2.29,5.99
75%,NaN,3.85,12.64
max,NaN,289.00,163333.33


# 5.3. Análisis del DataFrame de Mercadona

In [16]:
#Análisis general del DataFrame
df_dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723032 entries, 0 to 3723031
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   url              object 
 1   supermarket      object 
 2   category         object 
 3   name             object 
 4   description      float64
 5   price            float64
 6   reference_price  float64
 7   reference_unit   object 
 8   insert_date      object 
 9   product_id       object 
dtypes: float64(3), object(7)
memory usage: 284.0+ MB


In [17]:
#Para ver la cantidad de nulos
df_mercadona.isnull().sum()

url                     0
supermarket             0
category                0
name                    0
description        359186
price                   0
reference_price         0
reference_unit          0
insert_date             0
product_id              0
dtype: int64

In [18]:
#Analisis de cuántos produtos únicos ha tenido Mercadona desde el comienzo del registro de datos
df_mercadona_registros_inicial = df_mercadona.shape[0]
df_mercadona_productos_unicos_inicial = len(df_mercadona.product_id.unique())
df_mercadona_productos_unicos_inicial

9122

In [19]:
round(df_mercadona.describe(),2)

,price,reference_price
count,3855618.00,3855618.00
mean,5.91,6.74
std,74.97,12.17
min,0.08,0.00
25%,1.25,1.61
50%,1.99,4.00
75%,3.75,8.00
max,7915.05,472.97


# 5.4. Primeras conclusiones
De este primer análisis se puede observar que hay bastante trabajo que hacer con los datos, tanto por nulos como por outliers.

Se puede observar que hay outliers ya que el valor máximo tanto en precio como en precio de referencia son valores demasiado altos.

In [20]:
round(df_carrefour.describe(),2)

,description,price,reference_price
count,0.0,1967497.00,1967509.00
mean,NaN,5.09,22.40
std,NaN,9.31,608.89
min,NaN,0.14,0.01
25%,NaN,1.60,1.33
50%,NaN,2.81,3.85
75%,NaN,5.40,9.16
max,NaN,595.00,140000.00


In [21]:
round(df_dia.describe(),2)

,description,price,reference_price
count,0.0,3723026.00,3723026.00
mean,NaN,3.41,28.93
std,NaN,5.27,830.16
min,NaN,0.08,0.02
25%,NaN,1.49,2.38
50%,NaN,2.29,5.99
75%,NaN,3.85,12.64
max,NaN,289.00,163333.33


In [22]:
round(df_mercadona.describe(),2)

,price,reference_price
count,3855618.00,3855618.00
mean,5.91,6.74
std,74.97,12.17
min,0.08,0.00
25%,1.25,1.61
50%,1.99,4.00
75%,3.75,8.00
max,7915.05,472.97


In [23]:
print("Carrefour en este tiempo ha tenido", len(df_carrefour.product_id.unique()), "productos únicos")
print("Día en este tiempo ha tenido", len(df_dia.product_id.unique()), "productos únicos")
print("Mercadona en este tiempo ha tenido", len(df_mercadona.product_id.unique()), "productos únicos")

Carrefour en este tiempo ha tenido 15441 productos únicos
Día en este tiempo ha tenido 10641 productos únicos
Mercadona en este tiempo ha tenido 9122 productos únicos


# 6. Limpieza de los datos

Una posibilidad que se ha contemplado a la hora de realizar la limpieza de los datos era crea una 'clase' que contuviera todas las funciones para ir realizando la limpieza de las diferentes columnas.

Aunque trabajar con esa clase hubiera sido más "limpio" y resumido, se perdería la explicación paso a paso de la limpieza de cada columna así como de los distintos problemas y ejemplos que se han ido encontrado en cada caso. 

Además, el hecho de trabajar con 3 bloques de datos iguales, con las mismas columnas y tipos de datos, facilitaba esto. Sin embargo, se ha visto que aunque esta estructura era similiar, la limpieza de cada DataFrame tenía sus propias particularidades, por lo que era tambien interesante ir desgranándolas poco a poco y resolviendolas de forma independiente, aunque el trabajo quede más largo.

En caso de querer juntar todo el proceso en un solo código fuera de Jupyter Notebook, sí sería conveniente la implantación de esa 'clase' para optimizar el código.

Dicho esto, la limpieza va a consistir en corregir el nombre del supermercado (quitar el "-es"), catalogar mejor el producto de acuerdo a su categoría, trabajando la información de la columna Category e intentando asemejarla a una estructura de clase y subclase de acuerdo a la propia información del supermercado.

Otro punto interesantes es separar la fecha de recogida de la información en tres columnas (Día, Mes y Año) para poder realizar diferentes comparativas.

Tambien se trabajan los Outliers de las columnas Price y Reference Price, que tienen particularidades explicadas en cada uno de los apartados propios.

Otro arreglo que se realiza es intentar unificar las descripciones de los productos, que los datos vienen con un "product id" que hace referencia a varias descripciones en el campo "name", incluso haciendo referencia a diferentes productos (por ejemplo cambia el producto de 500g a 1kg pero lo agrupan dentro del mismo producto con el mismo id)

Todo esto se explica paso a paso en cada uno de los siguientes apartados.

# 6.1. Limpieza del DataFrame de Carrefour

In [24]:
df_carrefour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967509 entries, 0 to 1967508
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   url              object 
 1   supermarket      object 
 2   category         object 
 3   name             object 
 4   description      float64
 5   price            float64
 6   reference_price  float64
 7   reference_unit   object 
 8   insert_date      object 
 9   product_id       object 
dtypes: float64(3), object(7)
memory usage: 150.1+ MB


In [25]:
df_carrefour.isnull().sum()

url                      0
supermarket              0
category                 0
name                   253
description        1967509
price                   12
reference_price          0
reference_unit           0
insert_date              0
product_id               0
dtype: int64

### Posibles duplicados
Lo primero es eliminar posibles registros duplicados que pueda haber en el DataFrame

In [26]:
#Se eliminan las filas repetidas
df_carrefour= df_carrefour.drop_duplicates()

### Columna insert_date
Se convierte la columna a tipo DateTime

In [27]:
#Se convierte la columna insert_date a tipo DateTime
df_carrefour['insert_date'] = pd.to_datetime(df_carrefour['insert_date'])
df_carrefour.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1908378 entries, 0 to 1967508
Data columns (total 10 columns):
 #   Column           Dtype         
---  ------           -----         
 0   url              object        
 1   supermarket      object        
 2   category         object        
 3   name             object        
 4   description      float64       
 5   price            float64       
 6   reference_price  float64       
 7   reference_unit   object        
 8   insert_date      datetime64[ns]
 9   product_id       object        
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 160.2+ MB


### Columna supermarket
Se corrige el nombre del establecimiento, quitando el "-es" y poniendo la primera letra en mayúscula. Se guarda el resultado en la misma columna

In [28]:
df_carrefour['supermarket']=df_carrefour['supermarket'].str.split('-',0,expand=False)[0][0].title()
df_carrefour.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e
1,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de pollo sin gluten y sin lactosa ...,NaN,3.00,8.33,kg,2021-03-06,b02fc529217ae756011dad17df381298
2,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa vegetal de champiñones Suquipá 180 g,NaN,2.25,12.50,kg,2021-03-06,d488a137436e5d8e70b9a5a3cd572fe0


### Columna category
El objetivo es intentar limpiar esta categoría y separarla en las distintas categorías jerárquicas que tiene Carrefour para sus productos. Para ello es necesario utilizar un excel aparte que realiza la conversión.

Esta *transformación* ha tenido que ser elaborada manualmente al no haber un patrón válido o una forma sencilla de traducir la columna category del CSV a una tabla jerarquica de tres niveles con categoría principal y dos subcategorías.

Sin embargo, la relación e inclusión de ese excel, en la tabla que compone los datos del CSV está todo automatizado con Python.
Además, se realiza una revisión de las categorías, y en caso de aparecer una caterogría nueva, se pide introducir su "conversión" para almacenarla en el excel y poder continuar con el proceso sin errores.

#### Comprobación de categorías nuevas

In [29]:
#Primero se ve cuantos campos únicos tiene la columna category para dimensionar la tarea
print("Carrefour tiene", len(df_carrefour.category.unique()), "categorias únicas en el listado")

Carrefour tiene 588 categorias únicas en el listado


In [30]:
carrefour_cat_excel=pd.read_excel('./carrefour_category.xlsx')
carrefour_cat_excel.head()

,category,Categoria1,Subcategoria1,Subcategoria2
0,bebe_alimentacion_infantil_complementos_alimen...,Bebé,Alimentación Infantil,Complementos Alimenticios e Infusiones para Bebé
1,bebe_alimentacion_infantil_leches_infantiles,Bebé,Alimentación Infantil,Leches Infantiles Bebé
2,bebe_alimentacion_infantil_papillas_y_galletas,Bebé,Alimentación Infantil,Papillas y Galletas
3,bebe_alimentacion_infantil_postres_zumos_petit...,Bebé,Alimentación Infantil,"Postres, Zumos Petit y yogures Infantiles"
4,bebe_alimentacion_infantil_tarritos_y_platos_p...,Bebé,Alimentación Infantil,Tarritos y Platos Preparados


In [31]:
#Se comprueba que todas las categorías del DataFrame formado con los CSV están en el excel de transformación
#En caso de que falte alguna, se pide introducirla en el excel

#Lista vacía para acumular las categorías que falten
lista_cat_carrefour = []

#Variable de control
n = 0

while n<1:
    #Dentro del bucle se lee el excel que contiene la transformación de categorías, para ir comprobando que las nuevas categorias
    #Se han introducido y si van quedando nuevas sin introducir, por si falta más de una.
    
    #Se lee el excel
    carrefour_cat_excel=pd.read_excel('./carrefour_category.xlsx')
    
    #Se recorre la lista de categorías del DataFrame, convertido en lista
    for cat in list(df_carrefour.category.unique()):
        
        #Se busca si cada elemento de la lista de categorías está en el excel de transformación. En caso de no estar,
        #se añade a la lista vacía
        if cat not in list(carrefour_cat_excel.category):
            lista_cat_carrefour.append(cat)
            
    #Una vez terminada la comprobación, se ve si la lista está vacía o no. 
    #En caso de que no esté vacía muestra el primer elemento para pedir al usuario que introduzca dicha categoría
    if len(lista_cat_carrefour) > 0 :
        print("Hay categorías nuevas. Introduce la siguiente categoría: ")
        print(lista_cat_carrefour[0])
        print("\r")
        
        #Se pide al usuario que "traduzca" la cateogría
        Categoria1 = input("Introduce la primera categoría: ")
        Subcategoria1 = input("Introduce la segunda categoría: ")
        Subcategoria2 = input("Introduce la tercera categoría (si existe): ")
        
        #Se indica la ruta del excel para poder abrirlo y escribir en él las nuevas categorías.
        path = ('./carrefour_category.xlsx')
        book = load_workbook(path)
         
        ws = book.active
        
        #Se añaden las categorías introducidas por el usuario
        to_append = [lista_cat_carrefour[0], Categoria1, Subcategoria1, Subcategoria2]
        ws.append(to_append)
        
        #Se guardan los cambios en el excel y se cierra.
        book.save(path)
        book.close()
        
        #Se elimina la categoria introducida de la lista temporal.
        lista_cat_carrefour.remove(lista_cat_carrefour[0])
        
        #Se vuelve a comprobar la lista. Si está vacía se termina el bucle, sino se vuelve a solicitar introducir otra categoría
        if len(lista_cat_carrefour) == 0:
            print("\r")
            print("Categorías que faltaban introducidas. Ya no hay categorías nuevas")
            n=1
    
    #Si de primeras están todas las categorías, se sale del bucle y se indica que no hay categorías nuevas.
    else:
        print("No hay categorías nuevas")
        n=1

No hay categorías nuevas


#### Introducir las transformaciones en el DF de trabajo

Una vez realizada la comprobación de categorías nuevas, hay que introducir esta transformación en el DataFrame de trabajo

In [32]:
#Se incluyen las columnas de transformación de categorías en el DataFrame que contiene los datos de los CSV

#Primero se extrae el contenido del excel
carrefour_cat_excel=pd.read_excel('./carrefour_category.xlsx')

#Se realiza la unión con un merge()
df_carrefour= pd.merge(df_carrefour,carrefour_cat_excel,how="inner", on=["category", "category"])
df_carrefour.head(2)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2
0,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas
1,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de pollo sin gluten y sin lactosa ...,NaN,3.00,8.33,kg,2021-03-06,b02fc529217ae756011dad17df381298,El Mercado,Carnicería,Hamburguesas


In [33]:
#Se comprueba que no ha cambiado la longitud del DataFrame
len(df_carrefour.product_id)

1908378

A la hora de hacer este proceso, como hay categorias que no tienen los tres niveles es posible que se hayan introducido como NaN. Estos NaN hay que ponerlos a "vacío".

In [34]:
#Para ver la cantidad de nulos
if df_carrefour.Subcategoria1.isnull().sum() >0 or df_carrefour.Subcategoria2.isnull().sum() >0:
    df_carrefour["Subcategoria1"] = df_carrefour.Subcategoria1.fillna("")
    df_carrefour["Subcategoria2"] = df_carrefour.Subcategoria2.fillna("")
    print("Corregidos los campos NaN de las nuevas categorías introduciras")
       
else:
    print("No hay campos NaN en las nuevas categorías introduciras")

No hay campos NaN en las nuevas categorías introduciras


### Columna Price

In [35]:
#Se comprueba que hay campos vacíos en la columna Price
print("Hay", df_carrefour.price.isnull().sum(), "productos que no tienen precio en el DataFrame de Carrefour")

Hay 12 productos que no tienen precio en el DataFrame de Carrefour


In [36]:
#Primero sustituye el valor de los NaN por ""
df_carrefour["price"] = df_carrefour.price.fillna("")
df_carrefour.loc[(df_carrefour.price == "")]

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2
300566,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-17,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes
300572,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-18,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes
300575,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-19,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes
300577,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-20,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes
300583,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-21,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes
300586,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-22,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes
300591,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-23,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes
300596,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-24,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes
300599,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-25,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes
300603,https://www.carrefour.es/supermercado/whisky-j...,Carrefour,bebidas_alcoholes_todos_los_alcoholes,Whisky Jameson irlandés 70 cl.,NaN,,25.99,l,2022-07-26,c09040febafa3b18253fa796df590548,Bebidas,Alcoholes,Todos los Alcoholes


In [37]:
#Se crea una columna nueva en el dataframe que contenga la concatenación del ID, precio, precio de referencia y la fecha.
#Se utilizará para identificar la línea a reemplazar
df_carrefour["concat"] = df_carrefour["product_id"].str.cat(df_carrefour[["price", "reference_price"]].astype(str), sep="-")
df_carrefour.head(2)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2,concat
0,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas,fe8eee545a6dc5195dcc29e6e23ca97e-1.95-8.12
1,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de pollo sin gluten y sin lactosa ...,NaN,3.0,8.33,kg,2021-03-06,b02fc529217ae756011dad17df381298,El Mercado,Carnicería,Hamburguesas,b02fc529217ae756011dad17df381298-3.0-8.33


In [38]:
#Se obtienen los product_id únicos que tienen el precio vacío
lista_ID_null_price = list(df_carrefour.loc[(df_carrefour.price == "")].product_id.unique())

dicc_carrefour_price = {}

#Se utilizan estos ID para crear DF temporales e ir obteniendo diversos datos con los que obtener finalmente el precio
for producto in lista_ID_null_price:
    
    #DF temporal para ese producto y con el precio vacío.
    df_temp = df_carrefour.loc[(df_carrefour.product_id == producto) & (df_carrefour.price == "")]
    reference_price_temp = list(df_temp.reference_price)[0]
    clave = list(df_temp.concat)[0]
    
    #Segundo DF temporal para ese producto y con el precio de referencia obtenido y sin el precio vacío.
    df_temp2 = df_carrefour.loc[(df_carrefour.product_id == producto) & 
                                (df_carrefour.reference_price == reference_price_temp) & 
                                (df_carrefour.price != "")]
    
    price_temp = list(df_temp2.price)[0]
    
        
    #Se crea un diccionario con el campo concat y el precio
    dicc_carrefour_price[clave] = dicc_carrefour_price.get(clave, price_temp)

dicc_carrefour_price

{'c09040febafa3b18253fa796df590548--25.99': 18.19,
 '4dfc046c75551b4d2324070e00ae2a49--13.49': 13.49,
 '21db945fe921f68bc2fc6940a7372d19--2.29': 2.29}

In [39]:
#Se busca reemplazar los valores del campo price por los obtenidos en el diccionario anterior, relacionandolos con el campo concat
df_carrefour.loc[df_carrefour['concat'].isin(dicc_carrefour_price.keys()), 'price'] = df_carrefour['concat'].map(dicc_carrefour_price)

In [40]:
#Se comprueba que no hay campos vacíos en la columna Price
print("Hay", df_carrefour.price.isnull().sum(), "productos que no tienen precio en el DataFrame de Carrefour")

Hay 0 productos que no tienen precio en el DataFrame de Carrefour


In [41]:
#Se vuelve a poner la columna Price como float, que en el proceso se ha cambiado a Object
df_carrefour['price'] = df_carrefour['price'].astype('float64')

In [42]:
#Se borra la columna concat porque se va a necesitar crear otra para los outliers y así no tener varias
df_carrefour.drop(["concat"], axis = 1, inplace=True)

## Outliers en la columnas Price
Una vez se han rellenado las celdas vacías en la columna Price, hay que mirar si hay datos que son outliers.

El problema en este caso se produce al no poder realizar el procedimiento ni con el método del percentil ni con el método de la desviación estandar.

Por ejemplo, hay productos en los que se ha producido una subida o bajada de precio reciente, y apenas cuentan con un par de registros con el precio cambiado. Estos productos al tener una gran cantidad de registros con un precio anterior y apenas un par de registros con un precio nuevo, por cualquiera de los otros métodos (percentiles o desviación estandar) el registro con el precio nuevo se catalogaría como un Outlier, cuando en realidad es un precio que está bien. Tambien puede ser que un producto haya subido de precio de forma continua todos los meses, con este proceso los primeros y los últimos registros saldrían como Outliers cuando en realidad tampoco lo son. Este problema se da con los registros más recientes, pero si la base de datos se continúa actualizando estos registros que ahora se marcan como outlier se consolidarán con el tiempo cuando haya más registros de precios similares y en esas actualizaciones sucesivas dejaran de ser Outliers.

Para detectar realmente los precios desajustados y poder solventar el probrema de conjuntos de datos practicamente iguales y muy pocos diferentes, he optado por considerar como Outlier cualquier precio que sea la media de ese producto $\pm$ 0.49 veces la media. Este número viene después de hacer multitud de pruebas, ya que poniendo un número más bajo entraban como Outlier multitud de productos que no lo son y se quedaban fuera muchos que si lo son. Esto sigue pasando incluso con el límite establecido, pero en algún punto había que establecer el límite.

De esta forma, las pequeñas subidas o bajas de precios en productos que apenas han tenido variabilidad en el tiempo, o en productos que han tenido una subida más continuada, no se van a ver indentificados como Outliers.

En las celdas siguientes se puede por ejemplo un par de casos claros de outlier.

In [43]:
df_carrefour.loc[(df_carrefour.product_id=="d0d50e890e03829569a67a7b209d322c")
                 &(df_carrefour.insert_date>"2022-09-14")
                 &(df_carrefour.insert_date<"2022-09-27")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

#Como se puede ver, la fila central, marca un precio de 6.6€ cuando los registros anterior y posterior para el mismo producto
#tienen un precio de 1.85€. Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date
1829430,Queso camembert cremoso Carrefour 250 g,1.85,7.4,kg,2022-09-15
1829441,Queso camembert cremoso Carrefour 250 g,6.60,26.4,kg,2022-09-16
1829515,Queso camembert cremoso Carrefour 250 g,1.85,7.4,kg,2022-09-26


In [44]:
df_carrefour.loc[(df_carrefour.product_id=="a75c761c379f09706300372ffd295f01")
                 &(df_carrefour.insert_date>"2022-03-19")
                 &(df_carrefour.insert_date<"2022-03-25")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

#Como se puede ver, las dos ultimas filas, marcan un precio de 1.49€ que correspondería al precio si el producto fuera de 1kg
#pero según el nombre es un paquete de 500g. Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date
1875593,Pepino Carrefour a granel 500 g aprox,0.72,1.45,kg,2022-03-20
1875608,Pepino Carrefour a granel 500 g aprox,0.72,1.45,kg,2022-03-22
1875615,Pepino Carrefour a granel 500 g aprox,1.49,1.49,kg,2022-03-23
1875627,Pepino Carrefour a granel 500 g aprox,1.49,1.49,kg,2022-03-24


In [45]:
#Otro ejemplo puede ser, que ocurre lo mismo que en el caso anterior
df_carrefour.loc[(df_carrefour.product_id=="283db0320525f66e90bdfe543279a24d")
                 &(df_carrefour.insert_date>"2022-03-21")
                 &(df_carrefour.insert_date<"2022-03-25")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

,name,price,reference_price,reference_unit,insert_date
1767202,Preparado para cocido Carrefour 450 g aprox,2.34,5.2,kg,2022-03-22
1767208,Preparado para cocido Carrefour 450 g aprox,5.20,5.2,kg,2022-03-23
1767211,Preparado para cocido Carrefour 450 g aprox,5.20,5.2,kg,2022-03-24


Para detectar estos Outliers se tiene que calcular la media de los precios, agrupando los productos en base a una nueva columna que concatena el product_id y el nombre.

Una vez calculada la media para cada uno de estos "grupos", se calculan los limites superior e inferior que determina el limite a partir del cual el precio es un outlier.

Se introducen estos valores en el DF relacionandolos mediante el campo concatenado y se calcula si el precio es un outlier o no

In [46]:
#Se crea una nueva columna en el DF que concatene el product_id con el nombre y la unidad de referencia para agrupar por eso
df_carrefour["concat"] = df_carrefour["product_id"].str.cat(df_carrefour[["name", "reference_unit"]].astype(str), sep="-")

#se crea un DF que contenga para cada grupo la media
df_carrefour_price_mean = round(df_carrefour[["price","concat"]].groupby("concat", as_index=False).mean(),2)
df_carrefour_price_mean["price"] = df_carrefour_price_mean.price.fillna(0)
df_carrefour_price_mean.rename(columns={"price":"price_mean"}, inplace=True)

#Se calculan los límites superiores e inferiores
df_carrefour_price_mean["price_lower_limit"] = round(df_carrefour_price_mean["price_mean"] - 
                                                     0.49*df_carrefour_price_mean["price_mean"],2)
df_carrefour_price_mean["price_upper_limit"] = round(df_carrefour_price_mean["price_mean"] + 
                                                     0.49*df_carrefour_price_mean["price_mean"],2)

#Una vez creado el DataFrame, se introduce en el DF inicial
df_carrefour = pd.merge(df_carrefour, df_carrefour_price_mean,on='concat')

#Se crea una nueva columna que diga si el precio es un Outlier o no
df_carrefour["outlier_price"] = np.where(df_carrefour.price> df_carrefour.price_upper_limit,
                              "yes", np.where(df_carrefour.price < df_carrefour.price_lower_limit, "yes", "no"))
df_carrefour.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2,concat,price_mean,price_lower_limit,price_upper_limit,outlier_price
0,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas,fe8eee545a6dc5195dcc29e6e23ca97e-Hamburguesa d...,1.95,0.99,2.91,no
1,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-07,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas,fe8eee545a6dc5195dcc29e6e23ca97e-Hamburguesa d...,1.95,0.99,2.91,no
2,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-08,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas,fe8eee545a6dc5195dcc29e6e23ca97e-Hamburguesa d...,1.95,0.99,2.91,no


In [47]:
#Se indica cuantos outliers se han detectado con el método utilizado
print("Se han detectado", len(df_carrefour.loc[df_carrefour.outlier_price=="yes"]), "outliers en la columna price")

Se han detectado 2834 outliers en la columna price


In [48]:
#Se muestran estos outliers
df_carrefour.loc[df_carrefour.outlier_price=="yes"][["name", "product_id", "price", "reference_price", 
                                               "reference_unit", "insert_date", "price_mean",
                                               "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]

,name,product_id,price,reference_price,reference_unit,insert_date,price_mean,price_lower_limit,price_upper_limit,outlier_price
8196,Preparado para cocido Carrefour 450 g aprox,283db0320525f66e90bdfe543279a24d,5.20,5.20,kg,2022-03-23,2.28,1.16,3.40,yes
8197,Preparado para cocido Carrefour 450 g aprox,283db0320525f66e90bdfe543279a24d,5.20,5.20,kg,2022-03-24,2.28,1.16,3.40,yes
15096,Chirla nacional fresca 500 g,cdfd9358f173c5f7bb47a7f7954fdba6,5.90,11.80,kg,2021-03-31,3.94,2.01,5.87,yes
15143,Chirla nacional fresca 500 g,cdfd9358f173c5f7bb47a7f7954fdba6,5.95,11.90,kg,2022-01-06,3.94,2.01,5.87,yes
15885,Coquinas frescas 500 g,947ed1fe577a9e646af1ba1b391608bc,10.50,21.00,kg,2022-01-01,6.94,3.54,10.34,yes
...,...,...,...,...,...,...,...,...,...,...
1892679,Tomate azul a granel 500 g aprox,f008758f8fb4e13a828bb70ccbe3678d,3.69,3.69,kg,2022-07-22,2.30,1.17,3.43,yes
1892680,Tomate azul a granel 500 g aprox,f008758f8fb4e13a828bb70ccbe3678d,3.69,3.69,kg,2022-07-23,2.30,1.17,3.43,yes
1892681,Tomate azul a granel 500 g aprox,f008758f8fb4e13a828bb70ccbe3678d,3.69,3.69,kg,2022-07-24,2.30,1.17,3.43,yes
1892682,Tomate azul a granel 500 g aprox,f008758f8fb4e13a828bb70ccbe3678d,3.69,3.69,kg,2022-07-25,2.30,1.17,3.43,yes


Se comprueba que los ejemplos anteriores se han identificado bien como Outliers

In [49]:
df_carrefour.loc[(df_carrefour.product_id=="d0d50e890e03829569a67a7b209d322c")
                 &(df_carrefour.insert_date>"2022-09-14")
                 &(df_carrefour.insert_date<"2022-09-27")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", 
                                                "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]
#Como se puede ver, la fila central, marca un precio de 6.6€ cuando los registros anterior y posterior para el mismo producto
#tienen un precio de 1.85€. Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date,price_lower_limit,price_upper_limit,outlier_price
1878399,Queso camembert cremoso Carrefour 250 g,1.85,7.4,kg,2022-09-15,0.88,2.56,no
1878400,Queso camembert cremoso Carrefour 250 g,6.60,26.4,kg,2022-09-16,0.88,2.56,yes
1878401,Queso camembert cremoso Carrefour 250 g,1.85,7.4,kg,2022-09-26,0.88,2.56,no


In [50]:
df_carrefour.loc[(df_carrefour.product_id=="a75c761c379f09706300372ffd295f01")
                 &(df_carrefour.insert_date>"2022-03-19")
                 &(df_carrefour.insert_date<"2022-03-25")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", 
                                                "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,price_lower_limit,price_upper_limit,outlier_price
1740157,Pepino Carrefour a granel 500 g aprox,0.72,1.45,kg,2022-03-20,0.37,1.09,no
1740158,Pepino Carrefour a granel 500 g aprox,0.72,1.45,kg,2022-03-22,0.37,1.09,no
1740159,Pepino Carrefour a granel 500 g aprox,1.49,1.49,kg,2022-03-23,0.37,1.09,yes
1740160,Pepino Carrefour a granel 500 g aprox,1.49,1.49,kg,2022-03-24,0.37,1.09,yes


In [51]:
df_carrefour.loc[(df_carrefour.product_id=="283db0320525f66e90bdfe543279a24d")
                 &(df_carrefour.insert_date>"2022-03-21")
                 &(df_carrefour.insert_date<"2022-03-25")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", 
                                                "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,price_lower_limit,price_upper_limit,outlier_price
8195,Preparado para cocido Carrefour 450 g aprox,2.34,5.2,kg,2022-03-22,1.16,3.4,no
8196,Preparado para cocido Carrefour 450 g aprox,5.20,5.2,kg,2022-03-23,1.16,3.4,yes
8197,Preparado para cocido Carrefour 450 g aprox,5.20,5.2,kg,2022-03-24,1.16,3.4,yes


Como el número de outliers identificados es tan pequeño en comparación con el número de registros, se opta por la opción rápida que es eliminar todas las filas que contengan un Outlier

In [52]:
#Se eliminan las filas que tienen un yes en la columna Outlier
df_carrefour = df_carrefour.drop(df_carrefour[df_carrefour['outlier_price']=="yes"].index)

In [53]:
#Se comprueba que ya no quedan outliers
print("Quedan", len(df_carrefour.loc[df_carrefour.outlier_price=="yes"]), "outliers en la columna price")

Quedan 0 outliers en la columna price


In [54]:
#Se borra la columna concat que se necesita crear de nuevo para trabajar en los siguientes Outliers
df_carrefour.drop(["concat"], axis = 1, inplace=True)
df_carrefour.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2,price_mean,price_lower_limit,price_upper_limit,outlier_price
0,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas,1.95,0.99,2.91,no
1,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-07,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas,1.95,0.99,2.91,no
2,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-08,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas,1.95,0.99,2.91,no


El siguiente paso es comprobar si sigue habiendo outliers que no se hayan identificado con la fórmula anterior, como pasa más adelante con outliers del precio de referencia.

Se muestran los productos que tienen el precio más alto.

In [55]:
df_carrefour[["product_id","price"]].groupby("product_id", as_index=False).max().sort_values(["price"],ascending=False).head(5)

,product_id,price
443,07a4e9178ceb42cb5be73b99d19433ac,595.0
14718,f497f8533690dbf9211c51b4e88e8829,569.0
14264,ed479b0bd5afc3b5ca1ccb6d8392f7f5,486.0
1918,1fb6af387252cfee9a15c6e29a518d9f,459.0
4309,474f6f2fb3f25764a4f24b05bd480ce1,409.0


Viendo la tabla anterior, puede parecer de primeras que los precios más altos pueden ser Outliers. Sin embargo, haciendo un pequeño estudio se ve que no lo son:

* El primero es una barbacoa descatalogada (en el proceso de borrado de Outliers en el reference primer se borra este registro)

* El segundo es un Jamón de bellota ibérico 100% raza ibérica, que comprobando su precio en la [web](https://www.carrefour.es/supermercado/jamon-de-bellota-iberico-100-raza-iberica-pieza-7-kg-aprox/R-670002077/p) se ve que es correcto.

* El tercero es una barbacoa, que comprobando su precio en la [web](https://www.carrefour.es/barbacoa-vertical-con-chimena-y-ladrillos-refractarios/VC4A-3863044/p) se ve que también es correcto.

De esta forma, se descarta que los precios más altos en la columna price sean Outliers. Esto no quita para que sigua habiendo Outliers no detectados que hayn podido "escapar" de estos dos análisis

In [56]:
df_carrefour.loc[df_carrefour.product_id=="07a4e9178ceb42cb5be73b99d19433ac"][["name", "price", "reference_price", 
                                               "reference_unit"
                                              ]]

,name,price,reference_price,reference_unit
931383,Barbacoa de Pellets - 130x55x120 Cm / 54 kg,595.0,595.0,ud


In [57]:
df_carrefour.loc[df_carrefour.product_id=="f497f8533690dbf9211c51b4e88e8829"][["name", "price", "reference_price", 
                                               "reference_unit"
                                              ]].head(1)

,name,price,reference_price,reference_unit
1905076,Estuche de Jamón Ibérico de Bellota 100% Raza ...,569.0,71.12,kg


In [58]:
df_carrefour.loc[df_carrefour.product_id=="1fb6af387252cfee9a15c6e29a518d9f"][["name", "price", "reference_price", 
                                               "reference_unit"
                                              ]].head(1)

,name,price,reference_price,reference_unit
931390,Barbacoa Vertical con Chimena y Ladrillos Refr...,459.0,459.0,ud


## Outliers en la columnas Reference Price
Ahora es el turno de los outliers en el precio de referencia.

El procedimiento es el mismo que para los outliers en la columna price, así como tambien es igual su problemática y casuística.

En las celdas siguientes se puede por ejemplo un par de casos claros de outlier.

In [59]:
df_carrefour.loc[(df_carrefour.product_id=="34cc35798a0386be66cb178ae1926664")
                 &(df_carrefour.insert_date>"2021-04-19")
                 &(df_carrefour.insert_date<"2021-04-27")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

#Como se puede ver, las dos ultimas filas tienen el precio de referencia mal, ya que lo indica por unidad,
#pero el paquete tiene 4 unidades por lo que el precio entonces debería ser mayor.Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date
68133,Berlinas bombón Carrefour 4 ud,1.15,0.29,ud,2021-04-20
68134,Berlinas bombón Carrefour 4 ud,1.15,0.29,ud,2021-04-21
68135,Berlinas bombón Carrefour 4 ud,1.15,1.15,ud,2021-04-25
68136,Berlinas bombón Carrefour 4 ud,1.15,1.15,ud,2021-04-26


In [60]:
df_carrefour.loc[(df_carrefour.product_id=="1d5692b9b1a03bb08a18527a3a46980a")
                 &(df_carrefour.insert_date>"2021-12-02")
                 &(df_carrefour.insert_date<"2022-05-15")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

#Como se puede ver, las tres primeras filas, marcan un precio de referencia 4 veces más alto de lo que marcan el resto.
#Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date
302756,Yogur griego de vainilla Danone Oikos pack de ...,2.29,20.82,kg,2021-12-03
302757,Yogur griego de vainilla Danone Oikos pack de ...,2.29,20.82,kg,2021-12-22
302758,Yogur griego de vainilla Danone Oikos pack de ...,2.35,21.36,kg,2022-01-04
302759,Yogur griego de vainilla Danone Oikos pack de ...,2.35,5.34,kg,2022-01-29
302760,Yogur griego de vainilla Danone Oikos pack de ...,2.08,4.73,kg,2022-03-11
302761,Yogur griego de vainilla Danone Oikos pack de ...,2.35,5.34,kg,2022-04-09
302762,Yogur griego de vainilla Danone Oikos pack de ...,2.35,5.34,kg,2022-05-06


In [61]:
#Otro ejemplo puede ser, que se ve que el precio de referencia del atún no pude ser de 2 centimos el kg
df_carrefour.loc[(df_carrefour.product_id=="24003e647b709f9e8dde9d90084093de")
                 &(df_carrefour.insert_date>"2022-02-09")
                 &(df_carrefour.insert_date<"2022-02-15")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

,name,price,reference_price,reference_unit,insert_date
452244,Atún claro en aceite de girasol Calvo pack de ...,5.5,0.02,kg,2022-02-10
452245,Atún claro en aceite de girasol Calvo pack de ...,5.5,0.02,kg,2022-02-12
452246,Atún claro en aceite de girasol Calvo pack de ...,5.5,0.02,kg,2022-02-14


In [62]:
#Se crea una nueva columna en el DF que concatene el product_id con el nombre y la unidad de referencia para agrupar por eso
df_carrefour["concat"] = df_carrefour["product_id"].str.cat(df_carrefour[["name", "reference_unit"]].astype(str), sep="-")

#se crea un DF que contenga para cada grupo la media
df_carrefour_reference_price_mean = round(df_carrefour[["reference_price","concat"]].groupby("concat", as_index=False).mean(),2)
df_carrefour_reference_price_mean["reference_price"] = df_carrefour_reference_price_mean.reference_price.fillna(0)
df_carrefour_reference_price_mean.rename(columns={"reference_price":"reference_price_mean"}, inplace=True)

#Se calculan los límites superiores e inferiores
df_carrefour_reference_price_mean["reference_price_lower_limit"] = round(df_carrefour_reference_price_mean["reference_price_mean"] - 
                                                                         0.49*df_carrefour_reference_price_mean["reference_price_mean"],2)

df_carrefour_reference_price_mean["reference_price_upper_limit"] = round(df_carrefour_reference_price_mean["reference_price_mean"] + 
                                                                         0.49*df_carrefour_reference_price_mean["reference_price_mean"],2)

#Una vez creado el DataFrame, se introduce en el DF inicial
df_carrefour = pd.merge(df_carrefour, df_carrefour_reference_price_mean,on='concat')

#Se crea una nueva columna que diga si el precio es un Outlier o no
df_carrefour["outlier_reference_price"] = np.where(df_carrefour.reference_price > df_carrefour.reference_price_upper_limit,
                              "yes", np.where(df_carrefour.reference_price < df_carrefour.reference_price_lower_limit, "yes", "no"))
df_carrefour.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,...,Subcategoria2,price_mean,price_lower_limit,price_upper_limit,outlier_price,concat,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price
0,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,...,Hamburguesas,1.95,0.99,2.91,no,fe8eee545a6dc5195dcc29e6e23ca97e-Hamburguesa d...,8.12,4.14,12.1,no
1,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-07,fe8eee545a6dc5195dcc29e6e23ca97e,...,Hamburguesas,1.95,0.99,2.91,no,fe8eee545a6dc5195dcc29e6e23ca97e-Hamburguesa d...,8.12,4.14,12.1,no
2,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-08,fe8eee545a6dc5195dcc29e6e23ca97e,...,Hamburguesas,1.95,0.99,2.91,no,fe8eee545a6dc5195dcc29e6e23ca97e-Hamburguesa d...,8.12,4.14,12.1,no


In [63]:
#Se indica cuantos outliers se han detectado con el método utilizado
print("Se han detectado", len(df_carrefour.loc[df_carrefour.outlier_reference_price=="yes"]), "outliers en la columna reference price")

Se han detectado 4640 outliers en la columna reference price


In [64]:
#Se muestran estos outliers
df_carrefour.loc[df_carrefour.outlier_reference_price=="yes"][["name", "product_id", "price", "reference_price", 
                                               "reference_unit", "insert_date", "reference_price_mean",
                                               "reference_price_lower_limit", "reference_price_upper_limit",
                                                "outlier_reference_price"
                                              ]]

,name,product_id,price,reference_price,reference_unit,insert_date,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price
27680,La Gula del Norte con gambas al ajillo pack de...,fda516cb86d49b1b8579dfdf520f7aba,5.50,50.00,kg,2022-06-11,30.83,15.72,45.94,yes
27681,La Gula del Norte con gambas al ajillo pack de...,fda516cb86d49b1b8579dfdf520f7aba,5.50,50.00,kg,2022-06-12,30.83,15.72,45.94,yes
27682,La Gula del Norte con gambas al ajillo pack de...,fda516cb86d49b1b8579dfdf520f7aba,5.50,50.00,kg,2022-08-16,30.83,15.72,45.94,yes
27683,La Gula del Norte con gambas al ajillo pack de...,fda516cb86d49b1b8579dfdf520f7aba,5.50,50.00,kg,2022-09-29,30.83,15.72,45.94,yes
27684,La Gula del Norte con gambas al ajillo pack de...,fda516cb86d49b1b8579dfdf520f7aba,5.50,50.00,kg,2022-10-04,30.83,15.72,45.94,yes
...,...,...,...,...,...,...,...,...,...,...
1866871,"Sandía negra 5,5 Kg aprox",b5e463917f2cbaab62957a1201dbc44d,7.97,1.45,kg,2021-05-22,0.81,0.41,1.21,yes
1866872,"Sandía negra 5,5 Kg aprox",b5e463917f2cbaab62957a1201dbc44d,7.97,1.45,kg,2021-05-23,0.81,0.41,1.21,yes
1866873,"Sandía negra 5,5 Kg aprox",b5e463917f2cbaab62957a1201dbc44d,7.97,1.45,kg,2021-05-24,0.81,0.41,1.21,yes
1866874,"Sandía negra 5,5 Kg aprox",b5e463917f2cbaab62957a1201dbc44d,7.97,1.45,kg,2021-05-25,0.81,0.41,1.21,yes


Se comprueba que los ejemplos anteriores se han identificado bien como Outliers

In [65]:
df_carrefour.loc[(df_carrefour.product_id=="34cc35798a0386be66cb178ae1926664")
                 &(df_carrefour.insert_date>"2021-04-19")
                 &(df_carrefour.insert_date<"2021-04-27")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]]

#Como se puede ver, las dos ultimas filas tienen el precio de referencia mal, ya que lo indica por unidad,
#pero el paquete tiene 4 unidades por lo que el precio entonces debería ser mayor.Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
67957,Berlinas bombón Carrefour 4 ud,1.15,0.29,ud,2021-04-20,no
67958,Berlinas bombón Carrefour 4 ud,1.15,0.29,ud,2021-04-21,no
67959,Berlinas bombón Carrefour 4 ud,1.15,1.15,ud,2021-04-25,yes
67960,Berlinas bombón Carrefour 4 ud,1.15,1.15,ud,2021-04-26,yes


In [66]:
df_carrefour.loc[(df_carrefour.product_id=="1d5692b9b1a03bb08a18527a3a46980a")
                 &(df_carrefour.insert_date>"2021-12-02")
                 &(df_carrefour.insert_date<"2022-05-15")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]]

#Como se puede ver, las tres primeras filas, marcan un precio de referencia 4 veces más alto de lo que marcan el resto.
#Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
302088,Yogur griego de vainilla Danone Oikos pack de ...,2.29,20.82,kg,2021-12-03,yes
302089,Yogur griego de vainilla Danone Oikos pack de ...,2.29,20.82,kg,2021-12-22,yes
302090,Yogur griego de vainilla Danone Oikos pack de ...,2.35,21.36,kg,2022-01-04,yes
302091,Yogur griego de vainilla Danone Oikos pack de ...,2.35,5.34,kg,2022-01-29,no
302092,Yogur griego de vainilla Danone Oikos pack de ...,2.08,4.73,kg,2022-03-11,no
302093,Yogur griego de vainilla Danone Oikos pack de ...,2.35,5.34,kg,2022-04-09,no
302094,Yogur griego de vainilla Danone Oikos pack de ...,2.35,5.34,kg,2022-05-06,no


In [67]:
#Otro ejemplo puede ser, que se ve que el precio de referencia del atún no pude ser de 2 centimos el kg
df_carrefour.loc[(df_carrefour.product_id=="24003e647b709f9e8dde9d90084093de")
                 &(df_carrefour.insert_date>"2022-02-09")
                 &(df_carrefour.insert_date<"2022-02-15")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
451424,Atún claro en aceite de girasol Calvo pack de ...,5.5,0.02,kg,2022-02-10,yes
451425,Atún claro en aceite de girasol Calvo pack de ...,5.5,0.02,kg,2022-02-12,yes
451426,Atún claro en aceite de girasol Calvo pack de ...,5.5,0.02,kg,2022-02-14,yes


Como el número de outliers identificados es tan pequeño en comparación con el número de registros, se opta por la opción rápida que es eliminar todas las filas que contengan un Outlier

In [68]:
#Se eliminan las filas que tienen un yes en la columna Outlier
df_carrefour = df_carrefour.drop(df_carrefour[df_carrefour['outlier_reference_price']=="yes"].index)

In [69]:
#Se comprueba que ya no quedan outliers de los identificados con el método anterior
print("Quedan", len(df_carrefour.loc[df_carrefour.outlier_reference_price=="yes"]), "outliers en la columna reference price")

Quedan 0 outliers en la columna reference price


Se da la circunstancia de que hay productos que son claramente un Outlier, pero según el criterio no se han identificado como tal ya que todos los registros tienen el mismo valor.

Un ejemplo de este tipo de outlier es el siguiente, una bateria de cocina que tiene un precio por unidad de 140000€:

In [70]:
df_carrefour.loc[df_carrefour.reference_price==140000.00][["name", "price", "reference_price", 
                                               "reference_unit","outlier_reference_price"
                                              ]].head(2)

,name,price,reference_price,reference_unit,outlier_reference_price
1394304,Batería Acero Inoxidable MASTERPRO Triply 6 pzas,140.0,140000.0,ud,no
1394305,Batería Acero Inoxidable MASTERPRO Triply 6 pzas,140.0,140000.0,ud,no


Esto Outliers tambien hay que eliminarlos. Pero primero hay que identificar estos outlier que tienen un valor desorbitado por ser demasido elevado o un valor irrisorio por ser demasiado bajo.

In [71]:
df_carrefour[["product_id","reference_price"]].groupby("product_id", as_index=False).max().sort_values(["reference_price"],ascending=False)

,product_id,reference_price
11682,c39e65836c869cc8f3a43e944d19a4ed,140000.00
3349,3752b52580498e55d5310ba926562612,99000.00
1920,1fdafe2bc8b68e43b552a31b1d1cceaf,85000.00
4261,46cceb60dcaab89634ef658a0f76571b,75000.00
4753,4ed543514f801b2353da5ce35b0d4936,59000.00
...,...,...
9754,a3f134dac37a3473da93e2148f4f5aab,0.01
577,0a0048e3b4fdc1cdfc52069521d54607,0.01
14305,ee376d0c533c8fc3817403663d066428,0.01
542,095b0492b696c6f6b1ca9b0b9ede23da,0.01


En este caso tambien hay que poner un límite, asi que se determina que todo precio de referencia superior a 500 se marque como un Outlier.

En cuanto a límite inferior, hay muchos productos que su precio de referencia está indicado por unidad, mililitros, metros o lavados, y aunque sale un precio muy pequeño (inferior a 0.05) es un valor correcto.

Por tanto, no voy a indicar outliers con un precio de referencia muy pequeño, únicamente los grandes.

In [72]:
#Se identifican estos nuevos outliers
df_carrefour["outlier_reference_price"] = np.where(df_carrefour.reference_price >= 500,
                              "yes", "no")

In [73]:
#Se indica cuantos outliers se han detectado con esta nueva forma
print("Se han detectado", len(df_carrefour.loc[df_carrefour.outlier_reference_price=="yes"]), "nuevos outliers en la columna reference price")

Se han detectado 5666 nuevos outliers en la columna reference price


In [74]:
#Se eliminan estos outliers
df_carrefour = df_carrefour.drop(df_carrefour[df_carrefour['outlier_reference_price']=="yes"].index)

#Se comprueba que ya no quedan outliers
print("Quedan", len(df_carrefour.loc[df_carrefour.outlier_reference_price=="yes"]), "outliers en la columna reference price")

Quedan 0 outliers en la columna reference price


In [75]:
#Se borra la columna cocat que se necesita crear de nuevo para trabajar en la columna Name
df_carrefour.drop(["concat"], axis = 1, inplace=True)
df_carrefour.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,...,Subcategoria1,Subcategoria2,price_mean,price_lower_limit,price_upper_limit,outlier_price,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price
0,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,...,Carnicería,Hamburguesas,1.95,0.99,2.91,no,8.12,4.14,12.1,no
1,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-07,fe8eee545a6dc5195dcc29e6e23ca97e,...,Carnicería,Hamburguesas,1.95,0.99,2.91,no,8.12,4.14,12.1,no
2,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-08,fe8eee545a6dc5195dcc29e6e23ca97e,...,Carnicería,Hamburguesas,1.95,0.99,2.91,no,8.12,4.14,12.1,no


### Columna Name

Para corregir la columna Name se hace una agrupación de los productos por su id y por el precio de referencia, ya que hay muchos casos en los que se ha utilizado el mismo ID para productos que han cambiado de nombre al modificarse el propio producto. Una vez hecha esta agrupación mediante una nueva columna que concatena el ID y el predio de refrencia, se ordena además por la fecha más reciente, de tal forma que después se puede coger el nombre más reciente y ponerlo para cada agrupacion.

Esto se puede ver en el siguiente ejemplo:

In [76]:
df_carrefour.loc[(df_carrefour.product_id=="a75c761c379f09706300372ffd295f01")
                 &(df_carrefour.insert_date>"2022-03-19")
                 &(df_carrefour.insert_date<"2022-03-27")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", "product_id"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,product_id
1738082,Pepino Carrefour a granel 500 g aprox,0.72,1.45,kg,2022-03-20,a75c761c379f09706300372ffd295f01
1738083,Pepino Carrefour a granel 500 g aprox,0.72,1.45,kg,2022-03-22,a75c761c379f09706300372ffd295f01
1889498,Pepino Carrefour a granel 1 kg aprox,1.49,1.49,kg,2022-03-25,a75c761c379f09706300372ffd295f01
1889499,Pepino Carrefour a granel 1 kg aprox,1.49,1.49,kg,2022-03-26,a75c761c379f09706300372ffd295f01


In [77]:
#Se crea una nueva columna en el DF que concatene el product_id con el precio de referencia
df_carrefour["concat"] = df_carrefour["product_id"].str.cat(df_carrefour[["reference_price"]].astype(str), sep="-")

In [78]:
#Se ordena el DataFrame por concat y por fecha, para poder obtener el nombre más reciente para cada producto
df_carrefour=df_carrefour.sort_values(['concat', "insert_date"],ascending=False)

#Se crea una nueva columna que tiene para cada producto su descripción más actual.
#En caso de que la descripción más actual esté vacía, utiliza el campo name no vacío más actual.
df_carrefour["name_new"] = df_carrefour.groupby(by = "concat")['name'].transform('first')

#Se vuelve a ordenar el DataFrame por el índice
df_carrefour.sort_index(inplace=True)
df_carrefour.head()

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,...,price_mean,price_lower_limit,price_upper_limit,outlier_price,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price,concat,name_new
0,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,...,1.95,0.99,2.91,no,8.12,4.14,12.1,no,fe8eee545a6dc5195dcc29e6e23ca97e-8.12,Hamburguesa de vacuno raza frisona 240 g
1,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-07,fe8eee545a6dc5195dcc29e6e23ca97e,...,1.95,0.99,2.91,no,8.12,4.14,12.1,no,fe8eee545a6dc5195dcc29e6e23ca97e-8.12,Hamburguesa de vacuno raza frisona 240 g
2,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-08,fe8eee545a6dc5195dcc29e6e23ca97e,...,1.95,0.99,2.91,no,8.12,4.14,12.1,no,fe8eee545a6dc5195dcc29e6e23ca97e-8.12,Hamburguesa de vacuno raza frisona 240 g
3,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-09,fe8eee545a6dc5195dcc29e6e23ca97e,...,1.95,0.99,2.91,no,8.12,4.14,12.1,no,fe8eee545a6dc5195dcc29e6e23ca97e-8.12,Hamburguesa de vacuno raza frisona 240 g
4,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,Hamburguesa de vacuno raza frisona 240 g,NaN,1.95,8.12,kg,2021-03-10,fe8eee545a6dc5195dcc29e6e23ca97e,...,1.95,0.99,2.91,no,8.12,4.14,12.1,no,fe8eee545a6dc5195dcc29e6e23ca97e-8.12,Hamburguesa de vacuno raza frisona 240 g


In [79]:
print("Después de este paso quedan", df_carrefour.name_new.isnull().sum(), "registros sin nombre")

Después de este paso quedan 136 registros sin nombre


In [80]:
#Se sustituyen los NaN por ""
df_carrefour["name_new"] = df_carrefour.name_new.fillna("")

print("Esto son", len(df_carrefour.loc[df_carrefour.name_new==""].product_id.unique()), "productos únicos")

Esto son 8 productos únicos


In [81]:
#Se obtienen los product_id únicos que tienen el name vacío
lista_ID_null_name_new = list(df_carrefour.loc[(df_carrefour.name_new == "")].product_id.unique())

dicc_carrefour_name_new = {}

#Se utilizan estos ID para crear DF temporales e ir obteniendo diversos datos con los que obtener finalmente el precio
for producto in lista_ID_null_name_new:
    
    #DF temporal para ese producto y con el precio vacío.
    df_temp = df_carrefour.loc[(df_carrefour.product_id == producto) & (df_carrefour.name_new == "")]
    clave = list(df_temp.concat)[0]
    
    #Segundo DF temporal para ese producto y con el precio de referencia obtenido y sin el precio vacío.
    df_temp2 = df_carrefour.loc[(df_carrefour.product_id == producto) & 
                                (df_carrefour.name_new != "")]
    
    if len(list(df_temp2.name_new))>0:
        name_new_temp = list(df_temp2.name_new)[0]
    
        
        #Se crea un diccionario con el campo concat y el precio
        dicc_carrefour_name_new[clave] = dicc_carrefour_name_new.get(clave, name_new_temp)

dicc_carrefour_name_new

{'4df5adb2b2ebe635a5c83a98c1f4b3f0-40.86': 'Te Marrakesh Nescafe Dolce Gusto 16 Cápsulas',
 '1e35fcba9757d889f1b241413b680706-45.26': 'Gusanitos ecológico Smileat sin gluten 38 gr',
 '30ccb0484bd337734a028f451d1b707e-18.33': 'Patatas fritas sabor chilly & red pepper Tyrrells 40 g.',
 '05c478e893393538c12a2e053d63559d-29.01': 'Complemento Alimenticio desde 12 meses sabor fresa Pediasure 850 g.',
 '01884e5d26a3caef35038050f57d8aed-18.52': 'Cacao en cápsulas Nesquik Nescafe Dolce Gusto 16 unidades de 16 g.',
 '78d20051ec98b5ddfd6e271dc2a7f07b-0.34': 'Pants Huggies T6 (15-25 kg) 27 ud.',
 'c1937fc7c77bb4351a335f0f4d5017b2-22.83': 'Roll on de arnica y calamina Arnidol Pic 30 ml.',
 '40ed185a0c20d7b120c919c2b7fdbbea-23.6': 'Queso camembert de Normandie  Reflets de France  pieza 250 g'}

In [82]:
#Se busca reemplazar los valores del campo price por los obtenidos en el diccionario anterior, relacionandolos con el campo concat
df_carrefour.loc[df_carrefour['concat'].isin(dicc_carrefour_name_new.keys()), 'name_new'] = df_carrefour['concat'].map(dicc_carrefour_name_new)

In [83]:
print("Quedan", len(df_carrefour.loc[df_carrefour.name_new==""].product_id.unique()), "productos sin nombre")
print("Quedan", len(df_carrefour.loc[df_carrefour.name_new==""]), "registros sin nombre")

Quedan 4 productos sin nombre
Quedan 70 registros sin nombre


En este punto hay 47 registros, que corresponden a 3 productos, que no se han conseguido unir con el nombre, por lo que se eliminan del DataFrame

In [84]:
#Se eliminan las filas que tienen no tienen nombre
df_carrefour = df_carrefour.drop(df_carrefour[df_carrefour["name_new"]==""].index)

print("Quedan", len(df_carrefour.loc[df_carrefour.name_new==""].product_id.unique()), "productos sin nombre")
print("Quedan", len(df_carrefour.loc[df_carrefour.name_new==""]), "registros sin nombre")

Quedan 0 productos sin nombre
Quedan 0 registros sin nombre


### Ultimos pasos
Una vez terminado el trabajo en las columnas, rellenando los campos que faltaban, desglosando las categorías y unificando el campo "name", quedan dos pasos por hacer.
* Eliminar columnas: descripción que está vacía, y concat, category, lower_limit, upper_limit, outlier_price  y name que ya se ha trabajado con ellas.
* Reordenar las columnas del DataFrame.

In [85]:
#Eliminar columnas sobrantes
df_carrefour.drop(["concat", "name", "description",
                   "outlier_price", "price_lower_limit", "price_upper_limit",
                   "outlier_reference_price", "reference_price_lower_limit", "reference_price_upper_limit"],
                  axis = 1, inplace=True)

df_carrefour.head(2)

,url,supermarket,category,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2,price_mean,reference_price_mean,name_new
0,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,1.95,8.12,kg,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas,1.95,8.12,Hamburguesa de vacuno raza frisona 240 g
1,https://www.carrefour.es/supermercado/hamburgu...,Carrefour,el_mercado_carniceria_hamburguesas,1.95,8.12,kg,2021-03-07,fe8eee545a6dc5195dcc29e6e23ca97e,El Mercado,Carnicería,Hamburguesas,1.95,8.12,Hamburguesa de vacuno raza frisona 240 g


In [86]:
#Redordenar las columnas
df_carrefour = df_carrefour.reindex(columns=['supermarket',
                                             'name_new',
                                             'price',
                                             'reference_price',
                                             'reference_unit',
                                             'price_mean',
                                             'reference_price_mean',
                                             'category',
                                             'Categoria1',
                                             'Subcategoria1',
                                             'Subcategoria2',
                                             'insert_date',
                                             'product_id',
                                             'url'])
df_carrefour.head(2)

,supermarket,name_new,price,reference_price,reference_unit,price_mean,reference_price_mean,category,Categoria1,Subcategoria1,Subcategoria2,insert_date,product_id,url
0,Carrefour,Hamburguesa de vacuno raza frisona 240 g,1.95,8.12,kg,1.95,8.12,el_mercado_carniceria_hamburguesas,El Mercado,Carnicería,Hamburguesas,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,https://www.carrefour.es/supermercado/hamburgu...
1,Carrefour,Hamburguesa de vacuno raza frisona 240 g,1.95,8.12,kg,1.95,8.12,el_mercado_carniceria_hamburguesas,El Mercado,Carnicería,Hamburguesas,2021-03-07,fe8eee545a6dc5195dcc29e6e23ca97e,https://www.carrefour.es/supermercado/hamburgu...


Con esto queda "limpio" el DataFrame para poder empezar a trabjar con él y sacar más datos (aparte de la media ya sacada) y poder realizar hipótesis y obtener conclusiones de sus datos.

Después de la limpieza de los datos, el DataFrame queda de la siguiente forma

In [87]:
df_carrefour.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1895168 entries, 0 to 1905543
Data columns (total 14 columns):
 #   Column                Dtype         
---  ------                -----         
 0   supermarket           object        
 1   name_new              object        
 2   price                 float64       
 3   reference_price       float64       
 4   reference_unit        object        
 5   price_mean            float64       
 6   reference_price_mean  float64       
 7   category              object        
 8   Categoria1            object        
 9   Subcategoria1         object        
 10  Subcategoria2         object        
 11  insert_date           datetime64[ns]
 12  product_id            object        
 13  url                   object        
dtypes: datetime64[ns](1), float64(4), object(9)
memory usage: 216.9+ MB


In [88]:
df_carrefour.isnull().sum()

supermarket             0
name_new                0
price                   0
reference_price         0
reference_unit          0
price_mean              0
reference_price_mean    0
category                0
Categoria1              0
Subcategoria1           0
Subcategoria2           0
insert_date             0
product_id              0
url                     0
dtype: int64

In [89]:
print("Quedan", len(df_carrefour.product_id.unique()), "productos únicos y", len(df_carrefour.product_id), "registros en el DataFrame")
print("Se han eliminado",df_carrefour_productos_unicos_inicial-len(df_carrefour.product_id.unique()), "productos únicos y", df_carrefour_registros_inicial-len(df_carrefour.product_id), "registros en el DataFrame")

Quedan 15313 productos únicos y 1895168 registros en el DataFrame
Se han eliminado 128 productos únicos y 72341 registros en el DataFrame


In [90]:
round(df_carrefour.describe(),2)

,price,reference_price,price_mean,reference_price_mean
count,1895168.00,1895168.00,1895168.00,1895168.00
mean,5.08,7.50,5.08,7.50
std,9.31,12.50,9.28,12.47
min,0.14,0.01,0.14,0.01
25%,1.59,1.33,1.60,1.32
50%,2.80,3.84,2.80,3.85
75%,5.39,9.09,5.38,9.05
max,569.00,494.50,569.00,486.72


In [91]:
#El último paso, como se han ido borrando registros es reiniciar el índice.
df_carrefour.reset_index(drop=True,inplace=True)

# 6.2. Limpieza del DataFrame de Día

In [92]:
df_dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723032 entries, 0 to 3723031
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   url              object 
 1   supermarket      object 
 2   category         object 
 3   name             object 
 4   description      float64
 5   price            float64
 6   reference_price  float64
 7   reference_unit   object 
 8   insert_date      object 
 9   product_id       object 
dtypes: float64(3), object(7)
memory usage: 284.0+ MB


In [93]:
df_dia.isnull().sum()

url                      0
supermarket              0
category                 0
name                     0
description        3723032
price                    6
reference_price          6
reference_unit        5480
insert_date              0
product_id               0
dtype: int64

### Columna insert_date
Se convierte la columna a tipo DateTime

In [94]:
#Se convierte la columna insert_date a tipo DateTime
df_dia['insert_date'] = pd.to_datetime(df_dia['insert_date'])
df_dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723032 entries, 0 to 3723031
Data columns (total 10 columns):
 #   Column           Dtype         
---  ------           -----         
 0   url              object        
 1   supermarket      object        
 2   category         object        
 3   name             object        
 4   description      float64       
 5   price            float64       
 6   reference_price  float64       
 7   reference_unit   object        
 8   insert_date      datetime64[ns]
 9   product_id       object        
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 284.0+ MB


### Posibles duplicados
Lo primero es eliminar posibles registros duplicados que pueda haber en el DataFrame

In [95]:
#Se eliminan las filas repetidas
df_carrefour= df_carrefour.drop_duplicates()

### Columna supermarket
Se corrige el nombre del establecimiento, quitando el "-es" y poniendo la primera letra en mayúscula. Se guarda el resultado en la misma columna

In [96]:
df_dia['supermarket']=df_dia['supermarket'].str.split('-',0,expand=False)[0][0].title().replace("i", "í")
df_dia.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5
1,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,PUNTOMATIC detergente máquina blanco puro en p...,NaN,0.99,0.25,lavado,2021-03-06,e70c1708ea10f95a47e3ff9959a7e1fd
2,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,WIPP EXPRESS detergente máquina polvo maleta 3...,NaN,8.99,0.26,lavado,2021-03-06,23aed350daad23fe99f1d7dad9c5e0b3


### Columna category
Al igual que en el caso de Carrefour, en este punto el objetivo es intentar limpiar esta categoría y separarla en las distintas categorías jerárquicas que tiene Día para sus productos. Para ello es necesario utilizar un excel aparte que realiza la conversión.

Esta *transformación* ha tenido que ser elaborada manualmente al no haber un patrón válido o una forma sencilla de traducir la columna category del CSV a una tabla jerarquica de tres niveles con categoría principal y dos subcategorías.

Sin embargo, la relación e inclusión de ese excel, en la tabla que compone los datos del CSV está todo automatizado con Python.
Además, se realiza una revisión de las categorías, y en caso de aparecer una caterogría nueva, se pide introducir su "conversión" para almacenarla en el excel y poder continuar con el proceso sin errores.

Este Excel contiene más categorías de las que tiene el DataFrame actual, ya qu inicialmente se creó con los datos desde marzo de 2021. Como posteriormente se han limitado desde Noviembre de 2021 hay muchas categorías que ya no están, pero se dejan por si en el futuro vuelven a aparecer.

In [97]:
#Primero se ve cuantos campos únicos tiene la columna category para dimensionar la tarea
print("Día tiene", len(df_dia.category.unique()), "categorias únicas en el listado")

Día tiene 769 categorias únicas en el listado


In [98]:
#Se lee el excel que contiene la transformación de categorías
pd.read_excel('./dia_category.xlsx')

,category,Categoria1,Subcategoria1,Subcategoria2
0,al_dia_carne_cerdo,Al Día,Carne,Cerdo
1,al_dia_carne_conejo,Al Día,Carne,Conejo
2,al_dia_carne_mixto,Al Día,Carne,Mixto
3,al_dia_carne_pavo,Al Día,Carne,Pavo
4,al_dia_carne_pollo,Al Día,Carne,Pollo
...,...,...,...,...
764,platos_preparados_tortillas_tortillas,Platos Preparados,Tortillas,Tortillas
765,platos_preparados_verduras,Platos Preparados,Verduras,NaN
766,platos_preparados_verduras_verduras,Platos Preparados,Verduras,Verduras
767,productos_sin_gluten_productos_sin_gluten_prod...,Productos sin Gluten,Productos sin Gluten,NaN


In [99]:
#Se comprueba que todas las categorías del DataFrame formado con los CSV están en el excel de transformación
#En caso de que falte alguna, se pide introducirla en el excel

#Lista vacía para acumular las categorías que falten
lista_cat_dia = []

#Variable de control
n = 0

while n<1:
    #Dentro del bucle se lee el excel que contiene la transformación de categorías, para ir comprobando que las nuevas categorias
    #Se han introducido y si van quedando nuevas sin introducir, por si falta más de una.
    
    #Se lee el excel
    dia_cat_excel=pd.read_excel('./dia_category.xlsx')
    
    #Se recorre la lista de categorías del DataFrame, convertido en lista
    for cat in list(df_dia.category.unique()):
        
        #Se busca si cada elemento de la lista de categorías está en el excel de transformación. En caso de no estar,
        #se añade a la lista vacía
        if cat not in list(dia_cat_excel.category):
            lista_cat_dia.append(cat)
            
    #Una vez terminada la comprobación, se ve si la lista está vacía o no. 
    #En caso de que no esté vacía muestra el primer elemento para pedir al usuario que introduzca dicha categoría
    if len(lista_cat_dia) > 0 :
        print("Hay categorías nuevas. Introduce la siguiente categoría: ")
        print(lista_cat_dia[0])
        print("\r")
        
        #Se pide al usuario que "traduzca" la cateogría
        Categoria1 = input("Introduce la primera categoría: ")
        Subcategoria1 = input("Introduce la segunda categoría: ")
        Subcategoria2 = input("Introduce la tercera categoría (si existe): ")
        
        #Se indica la ruta del excel para poder abrirlo y escribir en él las nuevas categorías.
        path = ('./dia_category.xlsx')
        book = load_workbook(path)
         
        ws = book.active
        
        #Se añaden las categorías introducidas por el usuario
        to_append = [lista_cat_dia[0], Categoria1, Subcategoria1, Subcategoria2]
        ws.append(to_append)
        
        #Se guardan los cambios en el excel y se cierra.
        book.save(path)
        book.close()
        
        #Se elimina la categoria introducida de la lista temporal.
        lista_cat_dia.remove(lista_cat_dia[0])
        
        #Se vuelve a comprobar la lista. Si está vacía se termina el bucle, sino se vuelve a solicitar introducir otra categoría
        if len(lista_cat_dia) == 0:
            print("\r")
            print("Categorías que faltaban introducidas. Ya no hay categorías nuevas")
            n=1
    
    #Si de primeras están todas las categorías, se sale del bucle y se indica que no hay categorías nuevas.
    else:
        print("No hay categorías nuevas")
        n=1

No hay categorías nuevas


#### Introducir las transformaciones en el DF de trabajo
Una vez realizada la comprobación de categorías nuevas, hay que introducir esta transformación en el DataFrame de trabajo

In [100]:
#Se incluyen las columnas de transformación de categorías en el DataFrame que contiene los datos de los CSV

#Primero se extrae el contenido del excel
dia_cat_excel=pd.read_excel('./dia_category.xlsx')

#Se realiza la unión con un merge()
df_dia= pd.merge(df_dia,dia_cat_excel,how="inner", on=["category", "category"])
df_dia.head(4)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2
0,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas
1,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,PUNTOMATIC detergente máquina blanco puro en p...,NaN,0.99,0.25,lavado,2021-03-06,e70c1708ea10f95a47e3ff9959a7e1fd,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas
2,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,ARIEL Pods detergente máquina 3 en 1 original ...,NaN,5.99,0.33,lavado,2021-03-06,c366f5beb26f29ee6fa1ea3bc829ff13,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas
3,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,ARIEL Pods detergente máquina todo en uno en c...,NaN,5.99,0.33,lavado,2021-03-06,b9fce90098d39e3dbb83793129797650,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas


In [101]:
#Se comprueba que no ha cambiado la longitud del DataFrame
len(df_dia.product_id)

3723032

A la hora de hacer este proceso, como hay categorias que no tienen los tres niveles es posible que se hayan introducido como NaN. Estos NaN hay que ponerlos a "vacío".

In [102]:
#Para ver la cantidad de nulos
if df_dia.Subcategoria1.isnull().sum() >0 or df_dia.Subcategoria2.isnull().sum() >0:
    df_dia["Subcategoria1"] = df_dia.Subcategoria1.fillna("")
    df_dia["Subcategoria2"] = df_dia.Subcategoria2.fillna("")
    print("Corregidos los campos NaN de las nuevas categorías introduciras")
       
else:
    print("No hay campos NaN en las nuevas categorías introduciras")

Corregidos los campos NaN de las nuevas categorías introduciras


### Columna Price y Reference Price

In [103]:
#Se comprueba que hay campos vacíos en la columna Price
print("Hay", df_dia.price.isnull().sum(), "productos que no tienen precio en el DataFrame de Día")

Hay 6 productos que no tienen precio en el DataFrame de Día


In [104]:
#Primero sustituye el valor de los NaN por ""
df_dia["price"] = df_dia.price.fillna("")
df_dia["reference_price"] = df_dia.reference_price.fillna("")
df_dia.loc[(df_dia.price == "")]

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2
1118474,https://www.dia.es/compra-online/despensa/desa...,Día,despensa_desayunos_y_dulces_bolleria,BISCUITS GALICIA mini magdalenas con mantequil...,NaN,,,NaN,2022-06-27,d522a411cc1cc234dd353d4f14380733,Despensa,Desayunos y Dulces,Bollería
1118549,https://www.dia.es/compra-online/despensa/desa...,Día,despensa_desayunos_y_dulces_bolleria,BISCUITS GALICIA mini magdalenas con mantequil...,NaN,,,NaN,2022-06-28,d522a411cc1cc234dd353d4f14380733,Despensa,Desayunos y Dulces,Bollería
1388065,https://www.dia.es/compra-online/despensa/past...,Día,despensa_pastas_harinas_y_masas_pastas,DIA SELECCIÓN MUNDIAL tortiglioni paquete 500 gr,NaN,,,NaN,2022-06-27,61a283804a2289fbe661250c83c1a717,Despensa,"Pastas, Harinas y Masas",Pastas
1388111,https://www.dia.es/compra-online/despensa/past...,Día,despensa_pastas_harinas_y_masas_pastas,DIA SELECCIÓN MUNDIAL tortiglioni paquete 500 gr,NaN,,,NaN,2022-06-28,61a283804a2289fbe661250c83c1a717,Despensa,"Pastas, Harinas y Masas",Pastas
1817911,https://www.dia.es/compra-online/despensa/lact...,Día,despensa_lacteos_y_huevos_mantequilla_y_margarina,DIA margarina ligera barqueta 500 gr,NaN,,,NaN,2022-06-27,65f083cea3dd8750f4738e187ec0dbc4,Despensa,Lácteos y Huevos,Mantequilla y Margarina
1817928,https://www.dia.es/compra-online/despensa/lact...,Día,despensa_lacteos_y_huevos_mantequilla_y_margarina,DIA margarina ligera barqueta 500 gr,NaN,,,NaN,2022-06-28,65f083cea3dd8750f4738e187ec0dbc4,Despensa,Lácteos y Huevos,Mantequilla y Margarina


Como se puede observar, en este DataFrame, los prodcutos que no tienen precio y los que no tienen precio de referencia son los mismos, por lo que, dado el bajo número de productos que son, es más sencillo eliminarlos.

De todas formas, se deja por si acaso un código en caso de que únicamente falte el precio, como ha pasado en el DataFrame de Carrefour (reutilizando el código empleado en ese DF)

In [105]:
#Se eliminan las filas que tienen precio y precio de referencia vacias
df_dia = df_dia.drop(df_dia[(df_dia['price']=="") & 
                           (df_dia['reference_price']=="")].index)

#Se comprueba que no hay campos vacíos en la columna Price después de borrarlos
print("Hay", df_dia.price.isnull().sum(), "productos que no tienen precio en el DataFrame de Día")

Hay 0 productos que no tienen precio en el DataFrame de Día


El siguiente código es en caso de que después de hacer el paso anterior siga habiendo productos con Price nulos

In [106]:
if df_dia.price.isnull().sum() >0 :
    
    #Se crea una columna nueva en el dataframe que contenga la concatenación del ID, precio y precio de referencia.
    #Se utilizará para identificar la línea a reemplazar
    df_dia["concat"] = df_dia["product_id"].str.cat(df_dia[["price", "reference_price"]].astype(str), sep="-")
    df_dia.head(2)
    
    #Se obtienen los product_id únicos que tienen el precio vacío
    lista_dia_ID_null_price = list(df_dia.loc[(df_dia.price == "")].product_id.unique())

    dicc_dia_price = {}

    #Se utilizan estos ID para crear DF temporales e ir obteniendo diversos datos con los que obtener finalmente el precio
    for producto in lista_dia_ID_null_price:

        #DF temporal para ese producto y con el precio vacío.
        df_temp = df_dia.loc[(df_dia.product_id == producto) & (df_dia.price == "")]
        reference_price_temp = list(df_temp.reference_price)[0]
        clave = list(df_temp.concat)[0]

        #Segundo DF temporal para ese producto y con el precio de referencia obtenido y sin el precio vacío.
        df_temp2 = df_dia.loc[(df_dia.product_id == producto) & 
                              (df_dia.reference_price == reference_price_temp) & 
                              (df_dia.price != "")]

        price_temp = list(df_temp2.price)[0]


        #Se crea un diccionario con el campo concat y el precio
        dicc_dia_price[clave] = dicc_dia_price.get(clave, price_temp)

    #Se busca reemplazar los valores del campo price por los obtenidos en el diccionario anterior, relacionandolos 
    #con el campo concat
    df_dia.loc[df_dia['concat'].isin(dicc_dia_price.keys()), 'price'] = df_dia['concat'].map(dicc_dia_price)
    
    #Se comprueba que no hay campos vacíos en la columna Price
    print("Hay", df_dia.price.isnull().sum(), "productos que no tienen precio en el DataFrame de Día")
    
    #Se vuelve a poner la columna Price como float, que en el proceso se ha cambiado a Object
    df_dia['price'] = df_dia['price'].astype('float64')
    
    #Se borra la columna concat porque se va a necesitar crear otra para los outliers y así no tener varias
    df_dia.drop(["concat"], axis = 1, inplace=True)
    
else:
    None 

## Outliers en la columnas Price
Una vez se han rellenado las celdas vacías en la columna Price, hay que mirar si hay datos que son outliers.

El problema en este caso se produce al no poder realizar el procedimiento ni con el método del percentil ni con el método de la desviación estandar.

Por ejemplo, hay productos en los que se ha producido una subida o bajada de precio reciente, y apenas cuentan con un par de registros con el precio cambiado. Estos productos al tener una gran cantidad de registros con un precio anterior y apenas un par de registros con un precio nuevo, por cualquiera de los otros métodos (percentiles o desviación estandar) el registro con el precio nuevo se catalogaría como un Outlier, cuando en realidad es un precio que está bien. Tambien puede ser que un producto haya subido de precio de forma continua todos los meses, con este proceso los primeros y los últimos registros saldrían como Outliers cuando en realidad tampoco lo son. Este problema se da con los registros más recientes, pero si la base de datos se continúa actualizando estos registros que ahora se marcan como outlier se consolidarán con el tiempo cuando haya más registros de precios similares y en esas actualizaciones sucesivas dejaran de ser Outliers.

Para detectar realmente los precios desajustados y poder solventar el probrema de conjuntos de datos practicamente iguales y muy pocos diferentes, he optado por considerar como Outlier cualquier precio que sea la media de ese producto $\pm$ 0.49 veces la media. Este número viene después de hacer multitud de pruebas, ya que poniendo un número más bajo entraban como Outlier multitud de productos que no lo son y se quedaban fuera muchos que si lo son. Esto sigue pasando incluso con el límite establecido, pero en algún punto había que establecer el límite.

De esta forma, las pequeñas subidas o bajas de precios en productos que apenas han tenido variabilidad en el tiempo, o en productos que han tenido una subida más continuada, no se van a ver indentificados como Outliers.

En las celdas siguientes se puede por ejemplo un par de casos claros de outlier.

In [107]:
df_dia.loc[(df_dia.product_id=="f669141d2c2863cdbdecbe145f29a457")
          &(df_dia.insert_date>"2021-04-10")
          &(df_dia.insert_date<"2021-11-06")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

#En este caso se puede observar que dada la tendencia de los registros, los precios de 3€ son outliers

,name,price,reference_price,reference_unit,insert_date
29651,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",5.99,5.99,ud,2021-04-11
29745,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",5.99,5.99,ud,2021-04-12
1476591,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",3.0,3.0,ud,2021-06-11
1479087,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",3.0,3.0,ud,2021-07-06
1484795,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",5.99,5.99,ud,2021-08-31
1490687,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",5.99,5.99,ud,2021-10-29


In [108]:
df_dia.loc[(df_dia.product_id=="1a71d95d98adde5abbc04083c122813c")
          &(df_dia.insert_date>"2022-06-25")
          &(df_dia.insert_date<"2022-06-30")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

#Otro caso de Outlier, en el que el valor central es claramente un error

,name,price,reference_price,reference_unit,insert_date
390582,NIVEA Men active age crema de noche regenerado...,7.0,140.0,l,2022-06-27
390725,NIVEA Men active age crema de noche regenerado...,12.35,247.0,l,2022-06-28
390785,NIVEA Men active age crema de noche regenerado...,7.0,140.0,l,2022-06-29


In [109]:
df_dia.loc[(df_dia.product_id=="464c762f5b84b6366fe81eeffc387c47")
          &(df_dia.insert_date>"2022-08-27")
          &(df_dia.insert_date<"2022-09-03")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]
#Otro caso de Outlier, en el que el valor central es claramente un error

,name,price,reference_price,reference_unit,insert_date
2185993,COLON limpia lavadora caja 250 ml,4.69,18.76,l,2022-08-29
2186082,COLON limpia lavadora caja 250 ml,8.79,35.16,l,2022-08-30
2186300,COLON limpia lavadora caja 250 ml,4.99,19.96,l,2022-09-02


Para detectar estos Outliers se tiene que calcular la media de los precios, agrupando los productos en base a una nueva columna que concatena el product_id y el nombre.

Una vez calculada la media para cada uno de estos "grupos", se calculan los limites superior e inferior que determina el limite a partir del cual el precio es un outlier.

Se introducen estos valores en el DF relacionandolos mediante el campo concatenado y se calcula si el precio es un outlier o no

In [110]:
#Se crea una nueva columna en el DF que concatene el product_id con el nombre y la unidad de referencia para agrupar por eso
df_dia["concat"] = df_dia["product_id"].str.cat(df_dia[["name", "reference_unit"]].astype(str), sep="-")

#se crea un DF que contenga para cada grupo la media
df_dia_price_mean = round(df_dia[["price","concat"]].groupby("concat", as_index=False).mean(),2)
df_dia_price_mean["price"] = df_dia_price_mean.price.fillna(0)
df_dia_price_mean.rename(columns={"price":"price_mean"}, inplace=True)

#Se calculan los límites superiores e inferiores
df_dia_price_mean["price_lower_limit"] = round(df_dia_price_mean["price_mean"] - 0.49*df_dia_price_mean["price_mean"],2)
df_dia_price_mean["price_upper_limit"] = round(df_dia_price_mean["price_mean"] + 0.49*df_dia_price_mean["price_mean"],2)

#Una vez creado el DataFrame, se introduce en el DF inicial
df_dia = pd.merge(df_dia, df_dia_price_mean,on='concat')

#Se crea una nueva columna que diga si el precio es un Outlier o no
df_dia["outlier_price"] = np.where(df_dia.price> df_dia.price_upper_limit,
                              "yes", np.where(df_dia.price < df_dia.price_lower_limit, "yes", "no"))
df_dia.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2,concat,price_mean,price_lower_limit,price_upper_limit,outlier_price
0,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,834cbdc30d3f424ab90ea0e78c86a9a5-DIA detergent...,3.72,1.9,5.54,no
1,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-07,834cbdc30d3f424ab90ea0e78c86a9a5,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,834cbdc30d3f424ab90ea0e78c86a9a5-DIA detergent...,3.72,1.9,5.54,no
2,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-08,834cbdc30d3f424ab90ea0e78c86a9a5,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,834cbdc30d3f424ab90ea0e78c86a9a5-DIA detergent...,3.72,1.9,5.54,no


In [111]:
#Se indica cuantos outliers se han detectado con el método utilizado
print("Se han detectado", len(df_dia.loc[df_dia.outlier_price=="yes"]), "outliers en la columna price")

Se han detectado 2849 outliers en la columna price


In [112]:
#Se muestran estos outliers
df_dia.loc[df_dia.outlier_price=="yes"][["name", "price", "reference_price", "product_id", "outlier_price", 
                                        "price_mean","price_lower_limit","price_upper_limit"]]

,name,price,reference_price,product_id,outlier_price,price_mean,price_lower_limit,price_upper_limit
95186,Huevos frescos categoría A clase M estuche 12 uds,0.69,0.06,72dd8cd58a4fcd1194b159b5d3fd5802,yes,1.63,0.83,2.43
117956,DIA CAPRICHOSO copa de chocolate y nata pack 4...,1.59,3.46,80261030b763fe9c491f21dfa5b9c890,yes,1.01,0.52,1.50
117957,DIA CAPRICHOSO copa de chocolate y nata pack 4...,1.59,3.46,80261030b763fe9c491f21dfa5b9c890,yes,1.01,0.52,1.50
117958,DIA CAPRICHOSO copa de chocolate y nata pack 4...,1.59,3.46,80261030b763fe9c491f21dfa5b9c890,yes,1.01,0.52,1.50
117959,DIA CAPRICHOSO copa de chocolate y nata pack 4...,1.59,3.46,80261030b763fe9c491f21dfa5b9c890,yes,1.01,0.52,1.50
...,...,...,...,...,...,...,...,...
3673899,Melón piel de sapo unidad (4 Kg aprox.),3.96,0.99,cd3f6bc51b2d934c1e46b0136ed62779,yes,8.61,4.39,12.83
3675308,Naranja especial para zumo malla 4 Kg,5.99,1.5,b0e64d4c08e28c9c1c6bade21fe73cb8,yes,3.87,1.97,5.77
3675309,Naranja especial para zumo malla 4 Kg,5.99,1.5,b0e64d4c08e28c9c1c6bade21fe73cb8,yes,3.87,1.97,5.77
3675310,Naranja especial para zumo malla 4 Kg,5.99,1.5,b0e64d4c08e28c9c1c6bade21fe73cb8,yes,3.87,1.97,5.77


Se comprueba que los ejemplos anteriores se han identificado bien como Outliers

In [113]:
df_dia.loc[(df_dia.product_id=="f669141d2c2863cdbdecbe145f29a457")
          &(df_dia.insert_date>"2021-04-10")
          &(df_dia.insert_date<"2021-11-06")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", 
                                                "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]

#En este caso se puede observar que dada la tendencia de los registros, los precios de 3€ son outliers

,name,price,reference_price,reference_unit,insert_date,price_lower_limit,price_upper_limit,outlier_price
315318,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",5.99,5.99,ud,2021-04-11,3.01,8.79,no
315319,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",5.99,5.99,ud,2021-04-12,3.01,8.79,no
315320,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",3.0,3.0,ud,2021-06-11,3.01,8.79,yes
315321,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",3.0,3.0,ud,2021-07-06,3.01,8.79,yes
315322,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",5.99,5.99,ud,2021-08-31,3.01,8.79,no
315323,"LLONGUERAS tinte Marrón Glacé Nº 7,77 caja 1 ud",5.99,5.99,ud,2021-10-29,3.01,8.79,no


In [114]:
df_dia.loc[(df_dia.product_id=="1a71d95d98adde5abbc04083c122813c")
          &(df_dia.insert_date>"2022-06-25")
          &(df_dia.insert_date<"2022-06-30")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", 
                                                "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]

#Otro caso de Outlier, en el que el valor central es claramente un error

,name,price,reference_price,reference_unit,insert_date,price_lower_limit,price_upper_limit,outlier_price
1331380,NIVEA Men active age crema de noche regenerado...,7.0,140.0,l,2022-06-27,4.0,11.7,no
1331381,NIVEA Men active age crema de noche regenerado...,12.35,247.0,l,2022-06-28,4.0,11.7,yes
1331382,NIVEA Men active age crema de noche regenerado...,7.0,140.0,l,2022-06-29,4.0,11.7,no


In [115]:
df_dia.loc[(df_dia.product_id=="464c762f5b84b6366fe81eeffc387c47")
          &(df_dia.insert_date>"2022-08-27")
          &(df_dia.insert_date<"2022-09-03")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", 
                                                "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]
#Otro caso de Outlier, en el que el valor central es claramente un error

,name,price,reference_price,reference_unit,insert_date,price_lower_limit,price_upper_limit,outlier_price
501510,COLON limpia lavadora caja 250 ml,4.69,18.76,l,2022-08-29,2.32,6.78,no
501511,COLON limpia lavadora caja 250 ml,8.79,35.16,l,2022-08-30,2.32,6.78,yes
501512,COLON limpia lavadora caja 250 ml,4.99,19.96,l,2022-09-02,2.32,6.78,no


Como el número de outliers identificados es tan pequeño en comparación con el número de registros, se opta por la opción rápida que es eliminar todas las filas que contengan un Outlier

In [116]:
#Se eliminan las filas que tienen un yes en la columna Outlier
df_dia = df_dia.drop(df_dia[df_dia['outlier_price']=="yes"].index)

In [117]:
#Se comprueba que ya no quedan outliers
print("Quedan", len(df_dia.loc[df_dia.outlier_price=="yes"]), "outliers en la columna price")

Quedan 0 outliers en la columna price


In [118]:
#Se borra la columna concat que se necesita crear de nuevo para trabajar en los siguientes Outliers
df_dia.drop(["concat"], axis = 1, inplace=True)
df_dia.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2,price_mean,price_lower_limit,price_upper_limit,outlier_price
0,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,3.72,1.9,5.54,no
1,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-07,834cbdc30d3f424ab90ea0e78c86a9a5,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,3.72,1.9,5.54,no
2,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-08,834cbdc30d3f424ab90ea0e78c86a9a5,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,3.72,1.9,5.54,no


El siguiente paso es comprobar si sigue habiendo outliers que no se hayan identificado con la fórmula anterior, como pasa más adelante con outliers del precio de referencia.

Se muestran los productos que tienen el precio más alto.

In [119]:
df_dia[["product_id","price"]].groupby("product_id", as_index=False).max().sort_values(["price"],ascending=False).head(5)

,product_id,price
3149,4c51d734b50c903bc2130e9ab8f5875d,289.0
4287,6998dda2ec3a2b6155a6d9059c367a8c,149.0
4896,77c84532a4248f73fce137d52943dddb,135.0
10529,fd3169f99ca23bb458f84b2afd4f54b7,125.0
315,075c66884f227e395dc9db65c25fc226,119.0


In [120]:
df_dia.loc[(df_dia.product_id=="4c51d734b50c903bc2130e9ab8f5875d")&(df_dia.price==289.0)][["name", "price", "reference_price", 
                                               "reference_unit"
                                              ]].head(1)

,name,price,reference_price,reference_unit
3693340,NOBLEZA EXTREMEÑA jamón de bellota 100% Ibéric...,289.0,38.53,kg


In [121]:
df_dia.loc[(df_dia.product_id=="6998dda2ec3a2b6155a6d9059c367a8c")&(df_dia.price==149.0)][["name", "price", "reference_price", 
                                               "reference_unit"
                                              ]].head(1)

,name,price,reference_price,reference_unit
670849,MANUEL DE MONTEJO jamón de cebo Ibérico 50% pi...,149.0,149.0,kg


In [122]:
df_dia.loc[(df_dia.product_id=="77c84532a4248f73fce137d52943dddb")&(df_dia.price==135.0)][["name", "price", "reference_price", 
                                               "reference_unit"
                                              ]].head(1)

,name,price,reference_price,reference_unit
681822,LEGADO jamón de cebo Ibérico 50% raza ibérica ...,135.0,18.0,kg


Viendo la tabla anterior, puede parecer de primeras que los precios más altos pueden ser Outliers. Sin embargo, haciendo un pequeño estudio se ve que no lo son:

Ya que los tres primeros registros son jamones de alta calidad y por tanto de un precio elevado.

De esta forma, se descarta que los precios más altos en la columna price sean Outliers. Esto no quita para que sigua habiendo Outliers no detectados que hayn podido "escapar" de estos dos análisis

## Outliers en la columnas Reference Price
Ahora es el turno de los outliers en el precio de referencia.

El procedimiento es el mismo que para los outliers en la columna price, así como tambien es igual su problemática y casuística. La única diferencia es que en este caso, en base a diferentes pruebas el límite para el oulier se marca en $\pm$ 0.6 veces la media.

En las celdas siguientes se puede por ejemplo un par de casos claros de outlier.

In [123]:
df_dia.loc[(df_dia.product_id=="8532aa3085abd59247ba60d67e9013cc")
          &(df_dia.insert_date>"2022-10-18")
          &(df_dia.insert_date<"2022-10-27")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

#Este precio de referencia es un Outlier. Si se hace el cálculo, si el precio de venta es de 0.9 para 29g, el precio para 
#1kg sería de 31.03€, no puede ser de 1.29€.Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date
2968797,KIT KAT chocolatina santa paquete 29 gr,0.9,1.29,kg,2022-10-20
2968798,KIT KAT chocolatina santa paquete 29 gr,0.9,1.29,kg,2022-10-25


In [124]:
df_dia.loc[(df_dia.product_id=="f7cc8d73935dd23141431019e62c3c46")
          &(df_dia.insert_date>"2021-03-07")
          &(df_dia.insert_date<"2021-03-11")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]
#Este producto es un paquete que cotiene 12 unidade y marca el precio de referencia por unidad. No puede tener el mismo
#precio la unidad que el paquete completo si es de 12 unidades.Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date
443173,AUSONIA Discreet compresas de incontinencia ma...,6.99,6.99,ud,2021-03-08
443174,AUSONIA Discreet compresas de incontinencia ma...,6.99,6.99,ud,2021-03-09
443175,AUSONIA Discreet compresas de incontinencia ma...,6.99,6.99,ud,2021-03-10


In [125]:
df_dia.loc[(df_dia.product_id=="d46b430c810833e5efe5fcea3268543e")
          &(df_dia.insert_date>"2021-05-10")
          &(df_dia.insert_date<"2021-05-15")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]
#Este vino de casi 1L (75cl) se vende a 2.99€ pero el precio por litro es de 0.4€, lo que cláramente es un dato erróneo.

,name,price,reference_price,reference_unit,insert_date
3155940,RIO SECO vino blanco sauvignon DO Rueda botell...,2.99,0.4,l,2021-05-11
3155941,RIO SECO vino blanco sauvignon DO Rueda botell...,2.99,0.4,l,2021-05-12
3155942,RIO SECO vino blanco sauvignon DO Rueda botell...,2.99,0.4,l,2021-05-13
3155943,RIO SECO vino blanco sauvignon DO Rueda botell...,2.99,0.4,l,2021-05-14


In [126]:
#Se crea una nueva columna en el DF que concatene el product_id con el nombre y la unidad de referencia para agrupar por eso
df_dia["concat"] = df_dia["product_id"].str.cat(df_dia[["name", "reference_unit"]].astype(str), sep="-")


#se crea un DF que contenga para cada grupo la media
df_dia_reference_price_mean = round(df_dia[["reference_price","concat"]].groupby("concat", as_index=False).mean(),2)
df_dia_reference_price_mean["reference_price"] = df_dia_reference_price_mean.reference_price.fillna(0)
df_dia_reference_price_mean.rename(columns={"reference_price":"reference_price_mean"}, inplace=True)


#Se calculan los límites superiores e inferiores
df_dia_reference_price_mean["reference_price_lower_limit"] = round(df_dia_reference_price_mean["reference_price_mean"] - 
                                                                   0.6*df_dia_reference_price_mean["reference_price_mean"],2)

df_dia_reference_price_mean["reference_price_upper_limit"] = round(df_dia_reference_price_mean["reference_price_mean"] + 
                                                                   0.6*df_dia_reference_price_mean["reference_price_mean"],2)


#Una vez creado el DataFrame, se introduce en el DF inicial
df_dia = pd.merge(df_dia, df_dia_reference_price_mean,on='concat')

#Se crea una nueva columna que diga si el precio es un Outlier o no
df_dia["outlier_reference_price"] = np.where(df_dia.reference_price > df_dia.reference_price_upper_limit,
                              "yes", np.where(df_dia.reference_price < df_dia.reference_price_lower_limit, "yes", "no"))
df_dia.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,...,Subcategoria2,price_mean,price_lower_limit,price_upper_limit,outlier_price,concat,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price
0,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5,...,Detergente Maquina Tabletas,3.72,1.9,5.54,no,834cbdc30d3f424ab90ea0e78c86a9a5-DIA detergent...,0.18,0.07,0.29,no
1,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-07,834cbdc30d3f424ab90ea0e78c86a9a5,...,Detergente Maquina Tabletas,3.72,1.9,5.54,no,834cbdc30d3f424ab90ea0e78c86a9a5-DIA detergent...,0.18,0.07,0.29,no
2,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-08,834cbdc30d3f424ab90ea0e78c86a9a5,...,Detergente Maquina Tabletas,3.72,1.9,5.54,no,834cbdc30d3f424ab90ea0e78c86a9a5-DIA detergent...,0.18,0.07,0.29,no


In [127]:
#Se indica cuantos outliers se han detectado con el método utilizado
print("Se han detectado", len(df_dia.loc[df_dia.outlier_reference_price=="yes"]), "outliers en la columna reference price")

Se han detectado 11911 outliers en la columna reference price


In [128]:
#Se muestran estos outliers
df_dia.loc[df_dia.outlier_reference_price=="yes"][["name", "product_id", "price", "reference_price", 
                                               "reference_unit", "insert_date", "reference_price_mean",
                                               "reference_price_lower_limit", "reference_price_upper_limit",
                                                "outlier_reference_price"
                                              ]]

,name,product_id,price,reference_price,reference_unit,insert_date,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price
16301,DIA caldo de pollo estuche 24 pastillas,505bf4259dcab6360e7ba14bee314738,1.29,0.05,ud,2022-12-06,0.90,0.36,1.44,yes
16302,DIA caldo de pollo estuche 24 pastillas,505bf4259dcab6360e7ba14bee314738,1.29,0.05,ud,2022-12-07,0.90,0.36,1.44,yes
16303,DIA caldo de pollo estuche 24 pastillas,505bf4259dcab6360e7ba14bee314738,1.29,0.05,ud,2022-12-08,0.90,0.36,1.44,yes
84206,DIA té negro earl grey estuche 25 bolsitas,e7eb840aeb6ec46aee86c9dabe8ec763,1.15,1.15,ud,2021-03-06,0.70,0.28,1.12,yes
84207,DIA té negro earl grey estuche 25 bolsitas,e7eb840aeb6ec46aee86c9dabe8ec763,1.15,1.15,ud,2021-03-07,0.70,0.28,1.12,yes
...,...,...,...,...,...,...,...,...,...,...
3633293,Caldo desgrasado de verduras estuche 12 pastillas,fa0a46965678ddc00862d6ff7da57c72,0.7,0.06,ud,2022-12-07,0.54,0.22,0.86,yes
3633294,Caldo desgrasado de verduras estuche 12 pastillas,fa0a46965678ddc00862d6ff7da57c72,0.7,0.06,ud,2022-12-08,0.54,0.22,0.86,yes
3633459,Caldo desgrasado de carne estuche 12 pastillas,2f016a47953f4b872d39ed0cfe2b2198,0.7,0.06,ud,2022-12-06,0.54,0.22,0.86,yes
3633460,Caldo desgrasado de carne estuche 12 pastillas,2f016a47953f4b872d39ed0cfe2b2198,0.7,0.06,ud,2022-12-07,0.54,0.22,0.86,yes


Se comprueba que los ejemplos anteriores se han identificado bien como Outliers

In [129]:
df_dia.loc[(df_dia.product_id=="8532aa3085abd59247ba60d67e9013cc")
          &(df_dia.insert_date>"2022-10-18")
          &(df_dia.insert_date<"2022-10-27")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]]

#Este precio de referencia es un Outlier. Si se hace el cálculo, si el precio de venta es de 0.9 para 29g, el precio para 
#1kg sería de 31.03€, no puede ser de 1.29€.Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
2967000,KIT KAT chocolatina santa paquete 29 gr,0.9,1.29,kg,2022-10-20,yes
2967001,KIT KAT chocolatina santa paquete 29 gr,0.9,1.29,kg,2022-10-25,yes


In [130]:
df_dia.loc[(df_dia.product_id=="f7cc8d73935dd23141431019e62c3c46")
          &(df_dia.insert_date>"2021-03-07")
          &(df_dia.insert_date<"2021-03-11")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]]
#Este producto es un paquete que cotiene 12 unidade y marca el precio de referencia por unidad. No puede tener el mismo
#precio la unidad que el paquete completo si es de 12 unidades.Ese registro es claramente un Outlier

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
443119,AUSONIA Discreet compresas de incontinencia ma...,6.99,6.99,ud,2021-03-08,yes
443120,AUSONIA Discreet compresas de incontinencia ma...,6.99,6.99,ud,2021-03-09,yes
443121,AUSONIA Discreet compresas de incontinencia ma...,6.99,6.99,ud,2021-03-10,yes


In [131]:
df_dia.loc[(df_dia.product_id=="d46b430c810833e5efe5fcea3268543e")
          &(df_dia.insert_date>"2021-05-10")
          &(df_dia.insert_date<"2021-05-15")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
3153934,RIO SECO vino blanco sauvignon DO Rueda botell...,2.99,0.4,l,2021-05-11,yes
3153935,RIO SECO vino blanco sauvignon DO Rueda botell...,2.99,0.4,l,2021-05-12,yes
3153936,RIO SECO vino blanco sauvignon DO Rueda botell...,2.99,0.4,l,2021-05-13,yes
3153937,RIO SECO vino blanco sauvignon DO Rueda botell...,2.99,0.4,l,2021-05-14,yes


Como el número de outliers identificados es tan pequeño en comparación con el número de registros, se opta por la opción rápida que es eliminar todas las filas que contengan un Outlier

In [132]:
#Se eliminan las filas que tienen un yes en la columna Outlier
df_dia = df_dia.drop(df_dia[df_dia['outlier_reference_price']=="yes"].index)

In [133]:
#Se comprueba que ya no quedan outliers de los identificados con el método anterior
print("Quedan", len(df_dia.loc[df_dia.outlier_reference_price=="yes"]), "outliers en la columna reference price")

Quedan 0 outliers en la columna reference price


Se da la circunstancia de que hay productos que son claramente un Outlier, pero según el criterio no se han identificado como tal ya que todos los registros tienen el mismo valor.

Un ejemplo de este tipo de outlier es el siguiente, una bateria de cocina que tiene un precio por unidad de 163333.33€:

In [134]:
df_dia.loc[df_dia.reference_price==163333.33][["name", "price", "reference_price", 
                                               "reference_unit","outlier_reference_price"
                                              ]].head(2)

,name,price,reference_price,reference_unit,outlier_reference_price
2728598,TITTO BLUNI pack uomo colonia 75 ml + desodora...,12.25,163333.33,NaN,no
2728599,TITTO BLUNI pack uomo colonia 75 ml + desodora...,12.25,163333.33,NaN,no


Esto Outliers tambien hay que eliminarlos. Pero primero hay que identificar estos outlier que tienen un valor desorbitado por ser demasido elevado o un valor irrisorio por ser demasiado bajo.

In [135]:
df_dia[["product_id","reference_price"]].groupby("product_id", as_index=False).max().sort_values(["reference_price"],ascending=False)

,product_id,reference_price
10098,f3a85cfa00a29b3c55016fb4a90aae4e,163333.33
10009,f1978d3ea5c49c604b734356d862a808,17266.67
4079,64c96f07c16f81045c45e335056a9790,16633.33
2081,328c97a267af6a32a577670931c8e399,16402.99
5556,86e547c9045faa9ae517476e92aca39b,15642.86
...,...,...
76,01acc6afd9c2c362998372b2ab283831,0.06
1659,28f70c3074292993fbbfe6b890c9b7d5,0.05
8367,ca8cd78c47d0b9b9203471ba7b2cda15,0.05
1016,183461867a69f9a17850d46a5caf1173,0.05


En este caso tambien hay que poner un límite, asi que se determina que todo precio de referencia superior a 500 se marque como un Outlier.

En cuanto a límite inferior, hay muchos productos que su precio de referencia está indicado por unidad, mililitros, metros o lavados, y aunque sale un precio muy pequeño (inferior a 0.05) es un valor correcto.

Por tanto, no voy a indicar outliers con un precio de referencia muy pequeño, únicamente los grandes.

In [136]:
#Se identifican estos nuevos outliers
df_dia["outlier_reference_price"] = np.where(df_dia.reference_price >= 500,
                              "yes", "no")

In [137]:
#Se indica cuantos outliers se han detectado con esta nueva forma
print("Se han detectado", len(df_dia.loc[df_dia.outlier_reference_price=="yes"]), "nuevos outliers en la columna reference price")

Se han detectado 17370 nuevos outliers en la columna reference price


In [138]:
#Se eliminan estos outliers
df_dia = df_dia.drop(df_dia[df_dia['outlier_reference_price']=="yes"].index)

#Se comprueba que ya no quedan outliers
print("Quedan", len(df_dia.loc[df_dia.outlier_reference_price=="yes"]), "outliers en la columna reference price")

Quedan 0 outliers en la columna reference price


In [139]:
#Se borra la columna cocat que se necesita crear de nuevo para trabajar en la columna Name
df_dia.drop(["concat"], axis = 1, inplace=True)
df_dia.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,...,Subcategoria1,Subcategoria2,price_mean,price_lower_limit,price_upper_limit,outlier_price,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price
0,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5,...,Cuidado Ropa,Detergente Maquina Tabletas,3.72,1.9,5.54,no,0.18,0.07,0.29,no
1,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-07,834cbdc30d3f424ab90ea0e78c86a9a5,...,Cuidado Ropa,Detergente Maquina Tabletas,3.72,1.9,5.54,no,0.18,0.07,0.29,no
2,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,DIA detergente máquina en cápsulas 20 uds,NaN,3.79,0.19,lavado,2021-03-08,834cbdc30d3f424ab90ea0e78c86a9a5,...,Cuidado Ropa,Detergente Maquina Tabletas,3.72,1.9,5.54,no,0.18,0.07,0.29,no


## Columna reference unit

La columna referen_unit inicialmente tiene valores NaN que hay que corregir. Sin embargo, al limpiar los Outliers en los pasos anteriores, estos NaN desaparecen por lo que no es necesario realizar ninguna limpieza adicional.

In [140]:
df_dia.isnull().sum()

url                                  0
supermarket                          0
category                             0
name                                 0
description                    3690896
price                                0
reference_price                      0
reference_unit                       0
insert_date                          0
product_id                           0
Categoria1                           0
Subcategoria1                        0
Subcategoria2                        0
price_mean                           0
price_lower_limit                    0
price_upper_limit                    0
outlier_price                        0
reference_price_mean                 0
reference_price_lower_limit          0
reference_price_upper_limit          0
outlier_reference_price              0
dtype: int64

### Columna Name

Aunque esta columna pueda no tener NaN, es necesario unificar los nombres de los productos como se ha hecho en el DF de Carrefour.

In [141]:
#Se crea una nueva columna en el DF que concatene el product_id con el precio de referencia
df_dia["concat"] = df_dia["product_id"].str.cat(df_dia[["reference_price"]].astype(str), sep="-")

#Se ordena el DataFrame por concat y por fecha, para poder obtener el nombre más reciente para cada producto
df_dia=df_dia.sort_values(['concat', "insert_date"],ascending=False)

#Se crea una nueva columna que tiene para cada producto su descripción más actual.
#En caso de que la descripción más actual esté vacía, utiliza el campo name no vacío más actual.
df_dia["name_new"] = df_dia.groupby(by = "concat")['name'].transform('first')

#Se vuelve a ordenar el DataFrame por el índice
df_dia.sort_index(inplace=True)

print("Quedan", len(df_dia.loc[df_dia.name_new==""].product_id.unique()), "productos sin nombre")
print("Quedan", len(df_dia.loc[df_dia.name_new==""]), "registros sin nombre")

Quedan 0 productos sin nombre
Quedan 0 registros sin nombre


### Ultimos pasos
Una vez terminado el trabajo en las columnas, rellenando los campos que faltaban, desglosando las categorías y unificando el campo "name", quedan dos pasos por hacer.
* Eliminar columnas: descripción que está vacía, y concat, category, lower_limit, upper_limit, outlier_price  y name que ya se ha trabajado con ellas.
* Reordenar las columnas del DataFrame.

In [142]:
#Eliminar columnas sobrantes
df_dia.drop(["concat","name", "description",
            "outlier_price", "price_lower_limit", "price_upper_limit",
            "outlier_reference_price", "reference_price_lower_limit", "reference_price_upper_limit"],
            axis = 1, inplace=True)

df_dia.head(2)

,url,supermarket,category,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,Subcategoria2,price_mean,reference_price_mean,name_new
0,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,3.79,0.19,lavado,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,3.72,0.18,DIA detergente máquina en cápsulas 20 uds
1,https://www.dia.es/compra-online/productos/dro...,Día,drogueria_y_limpieza_cuidado_ropa__detergente_...,3.79,0.19,lavado,2021-03-07,834cbdc30d3f424ab90ea0e78c86a9a5,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,3.72,0.18,DIA detergente máquina en cápsulas 20 uds


In [143]:
#Redordenar las columnas
df_dia = df_dia.reindex(columns=['supermarket',
                                'name_new',
                                'price',
                                'reference_price',
                                'reference_unit',
                                'price_mean',
                                'reference_price_mean',
                                'category',
                                'Categoria1',
                                'Subcategoria1',
                                'Subcategoria2',
                                'insert_date',
                                 'product_id',
                                'url'])
df_dia.head(3)

,supermarket,name_new,price,reference_price,reference_unit,price_mean,reference_price_mean,category,Categoria1,Subcategoria1,Subcategoria2,insert_date,product_id,url
0,Día,DIA detergente máquina en cápsulas 20 uds,3.79,0.19,lavado,3.72,0.18,drogueria_y_limpieza_cuidado_ropa__detergente_...,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5,https://www.dia.es/compra-online/productos/dro...
1,Día,DIA detergente máquina en cápsulas 20 uds,3.79,0.19,lavado,3.72,0.18,drogueria_y_limpieza_cuidado_ropa__detergente_...,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,2021-03-07,834cbdc30d3f424ab90ea0e78c86a9a5,https://www.dia.es/compra-online/productos/dro...
2,Día,DIA detergente máquina en cápsulas 20 uds,3.79,0.19,lavado,3.72,0.18,drogueria_y_limpieza_cuidado_ropa__detergente_...,Droguería y Limpieza,Cuidado Ropa,Detergente Maquina Tabletas,2021-03-08,834cbdc30d3f424ab90ea0e78c86a9a5,https://www.dia.es/compra-online/productos/dro...


Con esto queda "limpio" el DataFrame para poder empezar a trabjar con él y sacar más datos (aparte de la media ya sacada) y poder realizar hipótesis y obtener conclusiones de sus datos.

Después de la limpieza de los datos, el DataFrame queda de la siguiente forma

In [144]:
df_dia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3690896 entries, 0 to 3720176
Data columns (total 14 columns):
 #   Column                Dtype         
---  ------                -----         
 0   supermarket           object        
 1   name_new              object        
 2   price                 object        
 3   reference_price       object        
 4   reference_unit        object        
 5   price_mean            float64       
 6   reference_price_mean  float64       
 7   category              object        
 8   Categoria1            object        
 9   Subcategoria1         object        
 10  Subcategoria2         object        
 11  insert_date           datetime64[ns]
 12  product_id            object        
 13  url                   object        
dtypes: datetime64[ns](1), float64(2), object(11)
memory usage: 422.4+ MB


In [145]:
df_dia.isnull().sum()

supermarket             0
name_new                0
price                   0
reference_price         0
reference_unit          0
price_mean              0
reference_price_mean    0
category                0
Categoria1              0
Subcategoria1           0
Subcategoria2           0
insert_date             0
product_id              0
url                     0
dtype: int64

In [146]:
print("Quedan", len(df_dia.product_id.unique()), "productos únicos y", len(df_dia.product_id), "registros en el DataFrame")
print("Se han eliminado",df_dia_productos_unicos_inicial-len(df_dia.product_id.unique()), "productos únicos y", df_dia_registros_inicial-len(df_dia.product_id), "registros en el DataFrame")

Quedan 10554 productos únicos y 3690896 registros en el DataFrame
Se han eliminado 87 productos únicos y 32136 registros en el DataFrame


In [147]:
round(df_dia.describe(),2)

,price_mean,reference_price_mean
count,3690896.00,3690896.00
mean,3.35,13.82
std,5.14,33.05
min,0.10,0.05
25%,1.49,2.38
50%,2.30,5.99
75%,3.81,12.53
max,289.00,586.69


In [148]:
#El último paso, como se han ido borrando registros es reiniciar el índice.
df_dia.reset_index(drop=True, inplace=True)

# 6.3. Limpieza del DataFrame de Mercadona

In [149]:
df_mercadona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3855618 entries, 0 to 3855617
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   url              object 
 1   supermarket      object 
 2   category         object 
 3   name             object 
 4   description      object 
 5   price            float64
 6   reference_price  float64
 7   reference_unit   object 
 8   insert_date      object 
 9   product_id       object 
dtypes: float64(2), object(8)
memory usage: 294.2+ MB


In [150]:
df_mercadona.isnull().sum()

url                     0
supermarket             0
category                0
name                    0
description        359186
price                   0
reference_price         0
reference_unit          0
insert_date             0
product_id              0
dtype: int64

### Columna insert_date
Se convierte la columna a tipo DateTime

In [151]:
#Se convierte la columna insert_date a tipo DateTime
df_mercadona['insert_date'] = pd.to_datetime(df_mercadona['insert_date'])
df_mercadona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3855618 entries, 0 to 3855617
Data columns (total 10 columns):
 #   Column           Dtype         
---  ------           -----         
 0   url              object        
 1   supermarket      object        
 2   category         object        
 3   name             object        
 4   description      object        
 5   price            float64       
 6   reference_price  float64       
 7   reference_unit   object        
 8   insert_date      datetime64[ns]
 9   product_id       object        
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 294.2+ MB


### Posibles duplicados
Lo primero es eliminar posibles registros duplicados que pueda haber en el DataFrame

In [152]:
#Se eliminan las filas repetidas
df_carrefour= df_carrefour.drop_duplicates()

### Columna supermarket
Se corrige el nombre del establecimiento, quitando el "-es" y poniendo la primera letra en mayúscula. Se guarda el resultado en la misma columna

In [153]:
df_mercadona['supermarket']=df_mercadona['supermarket'].str.split('-',0,expand=False)[0][0].title()
df_mercadona.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-06,6dfd858ef08db9863e83c462f25bd5df
1,https://tienda.mercadona.es/product/79052/caps...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Cápsulas kaptogras Deliplus,Caja,4.5,27.44,100g,2021-03-06,9def5af6e14252efe09c744a5248c575
2,https://tienda.mercadona.es/product/79621/comp...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Comprimidos vitaminas y minerales Deliplus,Caja,3.0,10.15,100g,2021-03-06,6ff9a5d970dfb2383f0742a2314fa972


### Columna category
Como en los dos otros supermercados, en este punto el objetivo es intentar limpiar esta categoría y separarla en las distintas categorías jerárquicas que tiene Mercadona para sus productos, que a diferencia de los anteriores solo tiene 2 niveles en vez de tres. Para ello es necesario utilizar un excel aparte que realiza la conversión.

Esta *transformación* ha tenido que ser elaborada manualmente al no haber un patrón válido o una forma sencilla de traducir la columna category del CSV a una tabla jerarquica de dos niveles con categoría principal y una subcategoría.

Sin embargo, la relación e inclusión de ese excel, en la tabla que compone los datos del CSV está todo automatizado con Python.
Además, se realiza una revisión de las categorías, y en caso de aparecer una caterogría nueva, se pide introducir su "conversión" para almacenarla en el excel y poder continuar con el proceso sin errores.

In [154]:
#Primero se ve cuantos campos únicos tiene la columna category para dimensionar la tarea
print("Mercadona tiene", len(df_mercadona.category.unique()), "categorias únicas en el listado")

Mercadona tiene 155 categorias únicas en el listado


In [155]:
#Se lee el excel que contiene la transformación de categorías
pd.read_excel('./mercadona_category.xlsx')

,category,Categoria1,Subcategoria1
0,aceite_especias_y_salsas_aceite_vinagre_y_sal,"Aceite, Especias y Salsas","Aceite, Vinagre y Sal"
1,aceite_especias_y_salsas_especias,"Aceite, Especias y Salsas",Especias
2,aceite_especias_y_salsas_mayonesa_ketchup_y_mo...,"Aceite, Especias y Salsas","Mayonesa, Kétchup y Mostaza"
3,aceite_especias_y_salsas_otras_salsas,"Aceite, Especias y Salsas",Otras Salsas
4,agua_y_refrescos_agua,Agua y Refrescos,Agua
...,...,...,...
150,postres_y_yogures_yogures_y_postres_infantiles,Postres y Yogures,Yogures y Postres Infantiles
151,zumos_fruta_variada,Zumos,Fruta Variada
152,zumos_melocoton_y_pina,Zumos,Melocotón y Piña
153,zumos_naranja,Zumos,Naranja


In [156]:
#Se comprueba que todas las categorías del DataFrame formado con los CSV están en el excel de transformación
#En caso de que falte alguna, se pide introducirla en el excel

#Lista vacía para acumular las categorías que falten
lista_cat_mercadona = []

#Variable de control
n = 0

while n<1:
    #Dentro del bucle se lee el excel que contiene la transformación de categorías, para ir comprobando que las nuevas categorias
    #Se han introducido y si van quedando nuevas sin introducir, por si falta más de una.
    
    #Se lee el excel
    mercadona_cat_excel=pd.read_excel('./mercadona_category.xlsx')
    
    #Se recorre la lista de categorías del DataFrame, convertido en lista
    for cat in list(df_mercadona.category.unique()):
        
        #Se busca si cada elemento de la lista de categorías está en el excel de transformación. En caso de no estar,
        #se añade a la lista vacía
        if cat not in list(mercadona_cat_excel.category):
            lista_cat_mercadona.append(cat)
            
    #Una vez terminada la comprobación, se ve si la lista está vacía o no. 
    #En caso de que no esté vacía muestra el primer elemento para pedir al usuario que introduzca dicha categoría
    if len(lista_cat_mercadona) > 0 :
        print("Hay categorías nuevas. Introduce la siguiente categoría: ")
        print(lista_cat_mercadona[0])
        print("\r")
        
        #Se pide al usuario que "traduzca" la cateogría. Mercadona solo tiene 2 niveles.
        Categoria1 = input("Introduce la primera categoría: ")
        Subcategoria1 = input("Introduce la segunda categoría (si existe): ")
        
        #Se indica la ruta del excel para poder abrirlo y escribir en él las nuevas categorías.
        path = ('./mercadona_category.xlsx')
        book = load_workbook(path)
         
        ws = book.active
        
        #Se añaden las categorías introducidas por el usuario
        to_append = [lista_cat_mercadona[0], Categoria1, Subcategoria1]
        ws.append(to_append)
        
        #Se guardan los cambios en el excel y se cierra.
        book.save(path)
        book.close()
        
        #Se elimina la categoria introducida de la lista temporal.
        lista_cat_mercadona.remove(lista_cat_mercadona[0])
        
        #Se vuelve a comprobar la lista. Si está vacía se termina el bucle, sino se vuelve a solicitar introducir otra categoría
        if len(lista_cat_mercadona) == 0:
            print("\r")
            print("Categorías que faltaban introducidas. Ya no hay categorías nuevas")
            n=1
    
    #Si de primeras están todas las categorías, se sale del bucle y se indica que no hay categorías nuevas.
    else:
        print("No hay categorías nuevas")
        n=1
    

No hay categorías nuevas


#### Introducir las transformaciones en el DF de trabajo
Una vez realizada la comprobación de categorías nuevas, hay que introducir esta transformación en el DataFrame de trabajo

In [157]:
#Se incluyen las columnas de transformación de categorías en el DataFrame que contiene los datos de los CSV

#Primero se extrae el contenido del excel
mercadona_cat_excel=pd.read_excel('./mercadona_category.xlsx')

#Se realiza la unión con un merge()
df_mercadona= pd.merge(df_mercadona,mercadona_cat_excel,how="inner", on=["category", "category"])
df_mercadona.head(4)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1
0,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.80,1.92,100g,2021-03-06,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia
1,https://tienda.mercadona.es/product/79052/caps...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Cápsulas kaptogras Deliplus,Caja,4.50,27.44,100g,2021-03-06,9def5af6e14252efe09c744a5248c575,Filoterapia y Parafarmacia,Fitoterapia
2,https://tienda.mercadona.es/product/79621/comp...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Comprimidos vitaminas y minerales Deliplus,Caja,3.00,10.15,100g,2021-03-06,6ff9a5d970dfb2383f0742a2314fa972,Filoterapia y Parafarmacia,Fitoterapia
3,https://tienda.mercadona.es/product/77168/caps...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Cápsulas dulces sueños Deliplus,Caja,3.95,30.38,100g,2021-03-06,86c0e5344f94e426991f8a08bfd45ce2,Filoterapia y Parafarmacia,Fitoterapia


A la hora de hacer este proceso, como hay categorias que no tienen los tres niveles es posible que se hayan introducido como NaN. Estos NaN hay que ponerlos a "vacío".

In [158]:
#Para ver la cantidad de nulos
if df_mercadona.Subcategoria1.isnull().sum()>0:
    df_mercadona["Subcategoria1"] = df_mercadona.Subcategoria1.fillna("")
    print("Corregidos los campos NaN de las nuevas categorías introduciras")
       
else:
    print("No hay campos NaN en las nuevas categorías introduciras")

No hay campos NaN en las nuevas categorías introduciras


In [159]:
#Además se convierte la columna Subcategoria1 en tipo Objetc que en el proceso se ha convertido en tipo float64
df_mercadona['Subcategoria1'] = df_mercadona['Subcategoria1'].astype('object')
df_mercadona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3855618 entries, 0 to 3855617
Data columns (total 12 columns):
 #   Column           Dtype         
---  ------           -----         
 0   url              object        
 1   supermarket      object        
 2   category         object        
 3   name             object        
 4   description      object        
 5   price            float64       
 6   reference_price  float64       
 7   reference_unit   object        
 8   insert_date      datetime64[ns]
 9   product_id       object        
 10  Categoria1       object        
 11  Subcategoria1    object        
dtypes: datetime64[ns](1), float64(2), object(9)
memory usage: 382.4+ MB


### Columna Price y Reference Price

En este DataFrame no hay elementos vacíos en las columnas Prices y Reference Price

In [160]:
#Se comprueba que hay campos vacíos en la columna Price
print("Hay", df_mercadona.price.isnull().sum(), "productos que no tienen precio en el DataFrame de Mercadona")
print("Hay", df_mercadona.reference_price.isnull().sum(), "productos que no tienen precio de referencia en el DataFrame de Mercadona")

Hay 0 productos que no tienen precio en el DataFrame de Mercadona
Hay 0 productos que no tienen precio de referencia en el DataFrame de Mercadona


## Outliers en la columnas Price
Una vez se han rellenado las celdas vacías en la columna Price, hay que mirar si hay datos que son outliers.

El problema en este caso se produce al no poder realizar el procedimiento ni con el método del percentil ni con el método de la desviación estandar.

Por ejemplo, hay productos en los que se ha producido una subida o bajada de precio reciente, y apenas cuentan con un par de registros con el precio cambiado. Estos productos al tener una gran cantidad de registros con un precio anterior y apenas un par de registros con un precio nuevo, por cualquiera de los otros métodos (percentiles o desviación estandar) el registro con el precio nuevo se catalogaría como un Outlier, cuando en realidad es un precio que está bien. Tambien puede ser que un producto haya subido de precio de forma continua todos los meses, con este proceso los primeros y los últimos registros saldrían como Outliers cuando en realidad tampoco lo son. Este problema se da con los registros más recientes, pero si la base de datos se continúa actualizando estos registros que ahora se marcan como outlier se consolidarán con el tiempo cuando haya más registros de precios similares y en esas actualizaciones sucesivas dejaran de ser Outliers.

Para detectar realmente los precios desajustados y poder solventar el probrema de conjuntos de datos practicamente iguales y muy pocos diferentes, he optado por considerar como Outlier cualquier precio que sea la media de ese producto $\pm$ 0.49 veces la media. Este número viene después de hacer multitud de pruebas, ya que poniendo un número más bajo entraban como Outlier multitud de productos que no lo son y se quedaban fuera muchos que si lo son. Esto sigue pasando incluso con el límite establecido, pero en algún punto había que establecer el límite.

De esta forma, las pequeñas subidas o bajas de precios en productos que apenas han tenido variabilidad en el tiempo, o en productos que han tenido una subida más continuada, no se van a ver indentificados como Outliers.

En las celdas siguientes se puede por ejemplo un par de casos claros de outlier.

In [161]:
df_mercadona.loc[(df_mercadona.product_id=="084bf0f75abdcc24764bda63729b48d4")
          &(df_mercadona.insert_date>"2022-03-19")
          &(df_mercadona.insert_date<"2022-03-23")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]


,name,price,reference_price,reference_unit,insert_date
1503404,Fiambre de pechuga de pavo Sajonia Serrano ahu...,2.99,8.082,kg,2022-03-20
1503426,Fiambre de pechuga de pavo Sajonia Serrano ahu...,8.58,23.190,kg,2022-03-21
1503474,Fiambre de pechuga de pavo Sajonia Serrano ahu...,3.17,8.568,kg,2022-03-22


In [162]:
df_mercadona.loc[(df_mercadona.product_id=="667d91c59017867249ddd2a749b94e60")
          &(df_mercadona.insert_date>"2022-03-16")
          &(df_mercadona.insert_date<"2022-03-22")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

,name,price,reference_price,reference_unit,insert_date
1188854,Foie gras de pato loncheado,7.45,49.667,kg,2022-03-17
1188902,Foie gras de pato loncheado,7.45,49.667,kg,2022-03-18
1188951,Foie gras de pato loncheado,51.25,341.667,kg,2022-03-19
1189012,Foie gras de pato loncheado,7.69,51.267,kg,2022-03-20
1189103,Foie gras de pato loncheado,7.69,51.267,kg,2022-03-21


In [163]:
df_mercadona.loc[(df_mercadona.product_id=="42019056bde3d64dfbb835754a7e7595")
          &(df_mercadona.insert_date>"2021-04-19")
          &(df_mercadona.insert_date<"2021-08-22")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]


,name,price,reference_price,reference_unit,insert_date
1450257,Manzana royal gala,0.33,1.65,kg,2021-04-20
1450313,Manzana royal gala,0.31,1.65,kg,2021-04-21
1456083,Manzana royal gala,1.25,1.69,kg,2021-08-18
1456115,Manzana royal gala,1.25,1.69,kg,2021-08-19
1456197,Manzana royal gala,0.35,1.69,kg,2021-08-20
1456259,Manzana royal gala,0.34,1.69,kg,2021-08-21


Para detectar estos Outliers se tiene que calcular la media de los precios, agrupando los productos en base a una nueva columna que concatena el product_id y el nombre.

Una vez calculada la media para cada uno de estos "grupos", se calculan los limites superior e inferior que determina el limite a partir del cual el precio es un outlier.

Se introducen estos valores en el DF relacionandolos mediante el campo concatenado y se calcula si el precio es un outlier o no

In [164]:
#Se crea una nueva columna en el DF que concatene el product_id con el nombre y la unidad de referencia para agrupar por eso
df_mercadona["concat"] = df_mercadona["product_id"].str.cat(df_mercadona[["name", "reference_unit"]].astype(str), sep="-")

#se crea un DF que contenga para cada grupo la media
df_mercadona_price_mean = round(df_mercadona[["price","concat"]].groupby("concat", as_index=False).mean(),2)
df_mercadona_price_mean["price"] = df_mercadona_price_mean.price.fillna(0)
df_mercadona_price_mean.rename(columns={"price":"price_mean"}, inplace=True)

#Se calculan los límites superiores e inferiores
df_mercadona_price_mean["price_lower_limit"] = round(df_mercadona_price_mean["price_mean"] -
                                                     0.49*df_mercadona_price_mean["price_mean"],2)

df_mercadona_price_mean["price_upper_limit"] = round(df_mercadona_price_mean["price_mean"] + 
                                                     0.49*df_mercadona_price_mean["price_mean"],2)

#Una vez creado el DataFrame, se introduce en el DF inicial
df_mercadona = pd.merge(df_mercadona, df_mercadona_price_mean,on='concat')

#Se crea una nueva columna que diga si el precio es un Outlier o no
df_mercadona["outlier_price"] = np.where(df_mercadona.price> df_mercadona.price_upper_limit,
                              "yes", np.where(df_mercadona.price < df_mercadona.price_lower_limit, "yes", "no"))
df_mercadona.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,concat,price_mean,price_lower_limit,price_upper_limit,outlier_price
0,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-06,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,6dfd858ef08db9863e83c462f25bd5df-Colagen sabor...,4.81,2.45,7.17,no
1,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-07,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,6dfd858ef08db9863e83c462f25bd5df-Colagen sabor...,4.81,2.45,7.17,no
2,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-08,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,6dfd858ef08db9863e83c462f25bd5df-Colagen sabor...,4.81,2.45,7.17,no


In [165]:
#Se indica cuantos outliers se han detectado con el método utilizado
print("Se han detectado", len(df_mercadona.loc[df_mercadona.outlier_price=="yes"]), "outliers en la columna price")

Se han detectado 2957 outliers en la columna price


In [166]:
#Se muestran estos outliers
df_mercadona.loc[df_mercadona.outlier_price=="yes"][["name", "price", "reference_price", "product_id", "outlier_price", 
                                        "price_mean","price_lower_limit","price_upper_limit"]]

,name,price,reference_price,product_id,outlier_price,price_mean,price_lower_limit,price_upper_limit
71964,Anís en grano Hacendado,1.80,36.000,7d0f10721c46dd94c4637efb54e45cf4,yes,1.14,0.58,1.70
71965,Anís en grano Hacendado,1.80,36.000,7d0f10721c46dd94c4637efb54e45cf4,yes,1.14,0.58,1.70
71966,Anís en grano Hacendado,1.80,36.000,7d0f10721c46dd94c4637efb54e45cf4,yes,1.14,0.58,1.70
71967,Anís en grano Hacendado,1.80,36.000,7d0f10721c46dd94c4637efb54e45cf4,yes,1.14,0.58,1.70
71968,Anís en grano Hacendado,1.80,36.000,7d0f10721c46dd94c4637efb54e45cf4,yes,1.14,0.58,1.70
...,...,...,...,...,...,...,...,...
3621116,Cerezas en almíbar denso Hacendado,1.95,21.667,2f953aeff16239a67c3cc1331c72f7b9,yes,1.15,0.59,1.71
3621117,Cerezas en almíbar denso Hacendado,1.95,21.667,2f953aeff16239a67c3cc1331c72f7b9,yes,1.15,0.59,1.71
3621118,Cerezas en almíbar denso Hacendado,1.95,21.667,2f953aeff16239a67c3cc1331c72f7b9,yes,1.15,0.59,1.71
3621119,Cerezas en almíbar denso Hacendado,1.95,21.667,2f953aeff16239a67c3cc1331c72f7b9,yes,1.15,0.59,1.71


Se comprueba que los ejemplos anteriores se han identificado bien como Outliers

In [167]:
df_mercadona.loc[(df_mercadona.product_id=="42019056bde3d64dfbb835754a7e7595")
          &(df_mercadona.insert_date>"2021-04-19")
          &(df_mercadona.insert_date<"2021-08-22")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", 
                                                "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]


,name,price,reference_price,reference_unit,insert_date,price_lower_limit,price_upper_limit,outlier_price
1473138,Manzana royal gala,0.33,1.65,kg,2021-04-20,0.17,0.51,no
1473139,Manzana royal gala,0.31,1.65,kg,2021-04-21,0.17,0.51,no
1473140,Manzana royal gala,1.25,1.69,kg,2021-08-18,0.17,0.51,yes
1473141,Manzana royal gala,1.25,1.69,kg,2021-08-19,0.17,0.51,yes
1473142,Manzana royal gala,0.35,1.69,kg,2021-08-20,0.17,0.51,no
1473143,Manzana royal gala,0.34,1.69,kg,2021-08-21,0.17,0.51,no


In [168]:
df_mercadona.loc[(df_mercadona.product_id=="084bf0f75abdcc24764bda63729b48d4")
          &(df_mercadona.insert_date>"2022-03-19")
          &(df_mercadona.insert_date<"2022-03-23")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", 
                                                "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,price_lower_limit,price_upper_limit,outlier_price
1531050,Fiambre de pechuga de pavo Sajonia Serrano ahu...,2.99,8.082,kg,2022-03-20,1.66,4.86,no
1531051,Fiambre de pechuga de pavo Sajonia Serrano ahu...,8.58,23.190,kg,2022-03-21,1.66,4.86,yes
1531052,Fiambre de pechuga de pavo Sajonia Serrano ahu...,3.17,8.568,kg,2022-03-22,1.66,4.86,no


In [169]:
df_mercadona.loc[(df_mercadona.product_id=="667d91c59017867249ddd2a749b94e60")
          &(df_mercadona.insert_date>"2022-03-16")
          &(df_mercadona.insert_date<"2022-03-22")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date", 
                                                "price_lower_limit", "price_upper_limit","outlier_price"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,price_lower_limit,price_upper_limit,outlier_price
1183151,Foie gras de pato loncheado,7.45,49.667,kg,2022-03-17,3.96,11.56,no
1183152,Foie gras de pato loncheado,7.45,49.667,kg,2022-03-18,3.96,11.56,no
1183153,Foie gras de pato loncheado,51.25,341.667,kg,2022-03-19,3.96,11.56,yes
1183154,Foie gras de pato loncheado,7.69,51.267,kg,2022-03-20,3.96,11.56,no
1183155,Foie gras de pato loncheado,7.69,51.267,kg,2022-03-21,3.96,11.56,no


Como el número de outliers identificados es tan pequeño en comparación con el número de registros, se opta por la opción rápida que es eliminar todas las filas que contengan un Outlier

In [170]:
#Se eliminan las filas que tienen un yes en la columna Outlier
df_mercadona = df_mercadona.drop(df_mercadona[df_mercadona['outlier_price']=="yes"].index)

In [171]:
#Se comprueba que ya no quedan outliers
print("Quedan", len(df_mercadona.loc[df_mercadona.outlier_price=="yes"]), "outliers en la columna price")

Quedan 0 outliers en la columna price


In [172]:
#Se vuelve a poner la columna Price y reference price como float, que en el proceso se ha cambiado a Object
df_mercadona['price'] = df_mercadona['price'].astype('float64')
df_mercadona['reference_price'] = df_mercadona['reference_price'].astype('float64')

In [173]:
#Se borra la columna concat que se necesita crear de nuevo para trabajar en los siguientes Outliers
df_mercadona.drop(["concat"], axis = 1, inplace=True)
df_mercadona.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,price_mean,price_lower_limit,price_upper_limit,outlier_price
0,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-06,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,4.81,2.45,7.17,no
1,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-07,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,4.81,2.45,7.17,no
2,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-08,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,4.81,2.45,7.17,no


El siguiente paso es comprobar si sigue habiendo outliers que no se hayan identificado con la fórmula anterior, como pasa más adelante con outliers del precio de referencia.

Se muestran los productos que tienen el precio más alto.

In [174]:
df_mercadona[["product_id","price"]].groupby("product_id", as_index=False).max().sort_values(["price"],ascending=False).head(20)

,product_id,price
5982,aa8de147e283c68f9b81ba55ef8bab49,7915.05
189,061743acc8d5c3b83c2b2578af5bc930,2965.05
6717,bf3cccbc52f49b3eab264679ed518d86,2074.05
2619,4a8c644f154514ed0ea037a647eeb62f,1960.20
8034,e24bb6aacfc47dc32d1ac70ba2358836,1633.50
5493,9d90ddbb171b5559fb818c838bf05049,1633.50
4757,87ec3ae6b79f7f8a98522f60a6a96537,1633.50
1949,3796c7ec2e71ea615360480bcbbe9bfd,1633.50
3281,5d2063c57035c998ac64bb0a52329e1a,1381.05
435,0cb5f24dff7d93467fc508b58121e435,1282.05


Analizando en detalle se ve que aquí, a diferencia de en los anteriores DataFrame, sí que hay precios mal.

Por ejemplo, este primer producto, aparte de ya no estar en la web de mercadona (accediendo por la url), se ve que tiene el mismo precio desorbitado para todos los registros, por lo que claramente este registro está mal. Analizando un poco más en detalle se ve que en realidad el precio debería de ser de 79.15€ ya que mercadona vende el prodcuto en cajas de 500g por unos 50€, por lo que se puede suponer que antes lo vendían en cajas de 1kg y por eso la url no funciona.

In [175]:
df_mercadona.loc[(df_mercadona.product_id=="aa8de147e283c68f9b81ba55ef8bab49")][["name", 
                                                "price", "reference_price", 
                                               "reference_unit", "price_mean", "insert_date"
                                              ]]


,name,price,reference_price,reference_unit,price_mean,insert_date
1714998,Carabinero congelado,7915.05,79.95,kg,7915.05,2021-05-21
1714999,Carabinero congelado,7915.05,79.95,kg,7915.05,2021-05-22
1715000,Carabinero congelado,7915.05,79.95,kg,7915.05,2021-05-23
1715001,Carabinero congelado,7915.05,79.95,kg,7915.05,2021-05-24
1715002,Carabinero congelado,7915.05,79.95,kg,7915.05,2021-05-25
...,...,...,...,...,...,...
1715097,Carabinero congelado,7915.05,79.95,kg,7915.05,2021-08-28
1715098,Carabinero congelado,7915.05,79.95,kg,7915.05,2021-08-29
1715099,Carabinero congelado,7915.05,79.95,kg,7915.05,2021-08-30
1715100,Carabinero congelado,7915.05,79.95,kg,7915.05,2021-08-31


El segundo producto es un caso similar al primero. La caja de *Alistado mediado* se vende actualemtne por unos 30€ por lo que es posible suponer que el precio de este producto es de 24.70€

In [176]:
df_mercadona.loc[(df_mercadona.product_id=="061743acc8d5c3b83c2b2578af5bc930")][["name", 
                                                "price", "reference_price", 
                                               "reference_unit", "price_mean"
                                              ]]
#Caso similar al anterior. La URL no lleva a ningun producto y buscando

,name,price,reference_price,reference_unit,price_mean
1704349,Alistado mediano congelado,2470.05,24.95,kg,2587.04
1704350,Alistado mediano congelado,2470.05,24.95,kg,2587.04
1704351,Alistado mediano congelado,2470.05,24.95,kg,2587.04
1704352,Alistado mediano congelado,2470.05,24.95,kg,2587.04
1704353,Alistado mediano congelado,2470.05,24.95,kg,2587.04
...,...,...,...,...,...
1705000,Alistado mediano congelado,2623.50,26.50,kg,2587.04
1705001,Alistado mediano congelado,2623.50,26.50,kg,2587.04
1705002,Alistado mediano congelado,2623.50,26.50,kg,2587.04
1705003,Alistado mediano congelado,2623.50,26.50,kg,2587.04


Este mismo patrón se repite en todos los productos que tienen un precio por encima de 500€. El primero que  no tiene un precio de más de 500€ es un jamón que sí tiene el precio correcto.

De esta forma, se pueden corregir estos precios sin necesidad de eliminar todos estos registros.

In [177]:
df_mercadona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3852661 entries, 0 to 3855617
Data columns (total 16 columns):
 #   Column             Dtype         
---  ------             -----         
 0   url                object        
 1   supermarket        object        
 2   category           object        
 3   name               object        
 4   description        object        
 5   price              float64       
 6   reference_price    float64       
 7   reference_unit     object        
 8   insert_date        datetime64[ns]
 9   product_id         object        
 10  Categoria1         object        
 11  Subcategoria1      object        
 12  price_mean         float64       
 13  price_lower_limit  float64       
 14  price_upper_limit  float64       
 15  outlier_price      object        
dtypes: datetime64[ns](1), float64(5), object(10)
memory usage: 499.7+ MB


In [178]:
#Se reemplaza el valor de los productos que tienen un valor de más de 500
df_mercadona["price"] = np.where(df_mercadona.price> 500, round(df_mercadona["price"]/100,2), df_mercadona.price)                             

In [179]:
#Se comprueba de nuevo la lista para ver si hay algún valor por encima de 500
df_mercadona.loc[(df_mercadona.price>500)][["name", 
                                                "price", "reference_price", 
                                               "reference_unit", "price_mean"
                                              ]]

,name,price,reference_price,reference_unit,price_mean


Como ha habido una correción en los precios, esta correción hay que aplicarla tambien en la columna price_mean, sino seguirán saliendo los valores anteriores.

Para ello, se borra la columna y se vuelve a obtener.

In [180]:
#Se borran las columnas
df_mercadona.drop(["price_mean", "price_lower_limit", "price_upper_limit"],axis = 1, inplace=True)

In [181]:
#Se crea una nueva columna en el DF que concatene el product_id con el nombre y la unidad de referencia para agrupar por eso
df_mercadona["concat"] = df_mercadona["product_id"].str.cat(df_mercadona[["name", "reference_unit"]].astype(str), sep="-")

#se crea un DF que contenga para cada grupo la media
df_mercadona_price_mean = round(df_mercadona[["price","concat"]].groupby("concat", as_index=False).mean(),2)
df_mercadona_price_mean["price"] = df_mercadona_price_mean.price.fillna(0)
df_mercadona_price_mean.rename(columns={"price":"price_mean"}, inplace=True)

#Se calculan los límites superiores e inferiores
df_mercadona_price_mean["price_lower_limit"] = round(df_mercadona_price_mean["price_mean"] -
                                                     0.49*df_mercadona_price_mean["price_mean"],2)

df_mercadona_price_mean["price_upper_limit"] = round(df_mercadona_price_mean["price_mean"] + 
                                                     0.49*df_mercadona_price_mean["price_mean"],2)

#Una vez creado el DataFrame, se introduce en el DF inicial
df_mercadona = pd.merge(df_mercadona, df_mercadona_price_mean,on='concat')

In [182]:
round(df_mercadona.describe(),2)

,price,reference_price,price_mean,price_lower_limit,price_upper_limit
count,3852661.00,3852661.00,3852661.00,3852661.00,3852661.00
mean,3.33,6.74,3.33,1.70,4.96
std,10.31,12.17,9.85,5.03,14.68
min,0.08,0.00,0.09,0.05,0.13
25%,1.25,1.61,1.26,0.64,1.88
50%,1.99,4.00,2.00,1.02,2.98
75%,3.75,8.00,3.72,1.90,5.54
max,490.05,472.97,466.51,237.92,695.10


## Outliers en la columnas Reference Price
Ahora es el turno de los outliers en el precio de referencia.

El procedimiento es el mismo que para los outliers en la columna price, así como tambien es igual su problemática y casuística. La única diferencia es que en este caso, en base a diferentes pruebas el límite para el oulier se marca en $\pm$ 0.6 veces la media.

En las celdas siguientes se puede por ejemplo un par de casos claros de outlier.

In [183]:
df_mercadona.loc[(df_mercadona.product_id=="44bd0538b2cd612d96f65d1993b8eb92")
          &(df_mercadona.insert_date>"2021-03-06")
          &(df_mercadona.insert_date<"2021-03-11")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]
#Las dos primeras filas tienen el precio de referencia igual que el precio normal. El producto no es de 1kg por lo que el 
#precio de referencia es un outlier

,name,price,reference_price,reference_unit,insert_date
1276515,Sangre de cerdo cocida,1.42,1.42,kg,2021-03-07
1276516,Sangre de cerdo cocida,1.42,1.42,kg,2021-03-08
1276517,Sangre de cerdo cocida,1.42,5.68,kg,2021-03-09
1276518,Sangre de cerdo cocida,1.42,5.68,kg,2021-03-10


In [184]:
df_mercadona.loc[(df_mercadona.product_id=="68264a79eb085d6cf8651bf662f4e84e")
          &(df_mercadona.insert_date>"2022-04-21")
          &(df_mercadona.insert_date<"2022-04-25")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]
#Este producto tambien tiene el precio de referencia mal, ya que lo pone para el paquete completo (el paquete tiene 4 cuchillas)
#por lo que en realidad deberia ser entre 4.

,name,price,reference_price,reference_unit,insert_date
146031,Maquinilla depilación desechable Colors Venus ...,3.45,3.45,ud,2022-04-22
146032,Maquinilla depilación desechable Colors Venus ...,3.45,3.45,ud,2022-04-23
146033,Maquinilla depilación desechable Colors Venus ...,3.45,3.45,ud,2022-04-24


In [185]:
df_mercadona.loc[(df_mercadona.product_id=="83f7a4a87582761bec26101fbf06122c")
          &(df_mercadona.insert_date>"2021-03-05")
          &(df_mercadona.insert_date<"2021-03-09")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date"
                                              ]]

#En este caso sucede lo mismo que en el caso anterior, en realiad el precio de referencia debería ser por goma no por paquete
#En el paquete van 5 gomas, por lo que el precio de referencia debería ser entre 5.

,name,price,reference_price,reference_unit,insert_date
3633324,Coleteros varios colores Deliplus,1.95,1.95,ud,2021-03-06
3633325,Coleteros varios colores Deliplus,1.95,1.95,ud,2021-03-07
3633326,Coleteros varios colores Deliplus,1.95,1.95,ud,2021-03-08


In [186]:
#Se crea una nueva columna en el DF que concatene el product_id con el nombre y la unidad de referencia para agrupar por eso
df_mercadona["concat"] = df_mercadona["product_id"].str.cat(df_mercadona[["name", "reference_unit"]].astype(str), sep="-")


#se crea un DF que contenga para cada grupo la media
df_mercadona_reference_price_mean = round(df_mercadona[["reference_price","concat"]].groupby("concat", as_index=False).mean(),2)
df_mercadona_reference_price_mean["reference_price"] = df_mercadona_reference_price_mean.reference_price.fillna(0)
df_mercadona_reference_price_mean.rename(columns={"reference_price":"reference_price_mean"}, inplace=True)


#Se calculan los límites superiores e inferiores
df_mercadona_reference_price_mean["reference_price_lower_limit"] = round(df_mercadona_reference_price_mean["reference_price_mean"] - 
                                                                   0.6*df_mercadona_reference_price_mean["reference_price_mean"],2)

df_mercadona_reference_price_mean["reference_price_upper_limit"] = round(df_mercadona_reference_price_mean["reference_price_mean"] + 
                                                                   0.6*df_mercadona_reference_price_mean["reference_price_mean"],2)


#Una vez creado el DataFrame, se introduce en el DF inicial
df_mercadona = pd.merge(df_mercadona, df_mercadona_reference_price_mean,on='concat')

#Se crea una nueva columna que diga si el precio es un Outlier o no
df_mercadona["outlier_reference_price"] = np.where(df_mercadona.reference_price > df_mercadona.reference_price_upper_limit,
                              "yes", np.where(df_mercadona.reference_price < df_mercadona.reference_price_lower_limit, "yes", "no"))
df_mercadona.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,...,Subcategoria1,outlier_price,concat,price_mean,price_lower_limit,price_upper_limit,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price
0,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-06,6dfd858ef08db9863e83c462f25bd5df,...,Fitoterapia,no,6dfd858ef08db9863e83c462f25bd5df-Colagen sabor...,4.81,2.45,7.17,1.92,0.77,3.07,no
1,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-07,6dfd858ef08db9863e83c462f25bd5df,...,Fitoterapia,no,6dfd858ef08db9863e83c462f25bd5df-Colagen sabor...,4.81,2.45,7.17,1.92,0.77,3.07,no
2,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-08,6dfd858ef08db9863e83c462f25bd5df,...,Fitoterapia,no,6dfd858ef08db9863e83c462f25bd5df-Colagen sabor...,4.81,2.45,7.17,1.92,0.77,3.07,no


In [187]:
#Se indica cuantos outliers se han detectado con el método utilizado
print("Se han detectado", len(df_mercadona.loc[df_mercadona.outlier_reference_price=="yes"]), "outliers en la columna reference price")

Se han detectado 3344 outliers en la columna reference price


In [188]:
#Se muestran estos outliers
df_mercadona.loc[df_mercadona.outlier_reference_price=="yes"][["name", "product_id", "price", "reference_price", 
                                               "reference_unit", "insert_date", "reference_price_mean",
                                               "reference_price_lower_limit", "reference_price_upper_limit",
                                                "outlier_reference_price"
                                              ]]

,name,product_id,price,reference_price,reference_unit,insert_date,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price
146027,Maquinilla depilación desechable Colors Venus ...,68264a79eb085d6cf8651bf662f4e84e,3.45,3.45,ud,2022-04-20,1.27,0.51,2.03,yes
146028,Maquinilla depilación desechable Colors Venus ...,68264a79eb085d6cf8651bf662f4e84e,3.45,3.45,ud,2022-04-21,1.27,0.51,2.03,yes
146029,Maquinilla depilación desechable Colors Venus ...,68264a79eb085d6cf8651bf662f4e84e,3.45,3.45,ud,2022-04-20,1.27,0.51,2.03,yes
146030,Maquinilla depilación desechable Colors Venus ...,68264a79eb085d6cf8651bf662f4e84e,3.45,3.45,ud,2022-04-21,1.27,0.51,2.03,yes
146031,Maquinilla depilación desechable Colors Venus ...,68264a79eb085d6cf8651bf662f4e84e,3.45,3.45,ud,2022-04-22,1.27,0.51,2.03,yes
...,...,...,...,...,...,...,...,...,...,...
3669888,Lima esmeril para uñas Deliplus,d84d8d52d1aaa4c0bd782d8e6534bff2,1.20,1.20,ud,2021-03-20,0.16,0.06,0.26,yes
3669889,Lima esmeril para uñas Deliplus,d84d8d52d1aaa4c0bd782d8e6534bff2,1.20,1.20,ud,2021-03-21,0.16,0.06,0.26,yes
3669890,Lima esmeril para uñas Deliplus,d84d8d52d1aaa4c0bd782d8e6534bff2,1.20,1.20,ud,2021-03-22,0.16,0.06,0.26,yes
3669891,Lima esmeril para uñas Deliplus,d84d8d52d1aaa4c0bd782d8e6534bff2,1.20,1.20,ud,2021-03-23,0.16,0.06,0.26,yes


Se comprueba que los ejemplos anteriores se han identificado bien como Outliers

In [189]:
df_mercadona.loc[(df_mercadona.product_id=="44bd0538b2cd612d96f65d1993b8eb92")
          &(df_mercadona.insert_date>"2021-03-06")
          &(df_mercadona.insert_date<"2021-03-11")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
1276515,Sangre de cerdo cocida,1.42,1.42,kg,2021-03-07,yes
1276516,Sangre de cerdo cocida,1.42,1.42,kg,2021-03-08,yes
1276517,Sangre de cerdo cocida,1.42,5.68,kg,2021-03-09,no
1276518,Sangre de cerdo cocida,1.42,5.68,kg,2021-03-10,no


In [190]:
df_mercadona.loc[(df_mercadona.product_id=="68264a79eb085d6cf8651bf662f4e84e")
          &(df_mercadona.insert_date>"2022-04-21")
          &(df_mercadona.insert_date<"2022-04-25")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
146031,Maquinilla depilación desechable Colors Venus ...,3.45,3.45,ud,2022-04-22,yes
146032,Maquinilla depilación desechable Colors Venus ...,3.45,3.45,ud,2022-04-23,yes
146033,Maquinilla depilación desechable Colors Venus ...,3.45,3.45,ud,2022-04-24,yes


In [191]:
df_mercadona.loc[(df_mercadona.product_id=="83f7a4a87582761bec26101fbf06122c")
          &(df_mercadona.insert_date>"2021-03-05")
          &(df_mercadona.insert_date<"2021-03-09")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]]

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
3633324,Coleteros varios colores Deliplus,1.95,1.95,ud,2021-03-06,yes
3633325,Coleteros varios colores Deliplus,1.95,1.95,ud,2021-03-07,yes
3633326,Coleteros varios colores Deliplus,1.95,1.95,ud,2021-03-08,yes


Como el número de outliers identificados es tan pequeño en comparación con el número de registros, se opta por la opción rápida que es eliminar todas las filas que contengan un Outlier

In [192]:
#Se eliminan las filas que tienen un yes en la columna Outlier
df_mercadona = df_mercadona.drop(df_mercadona[df_mercadona['outlier_reference_price']=="yes"].index)

In [193]:
#Se comprueba que ya no quedan outliers de los identificados con el método anterior
print("Quedan", len(df_mercadona.loc[df_mercadona.outlier_reference_price=="yes"]), "outliers en la columna reference price")

Quedan 0 outliers en la columna reference price


A diferencia de los otros DataFrame, en este después del proceso de limpieza hasta el momento no hay precios de referencia desmesurados.

Los más altos rondan los 450€ que analizandolos en detalle, son Azafran, por lo que el precio está bien.

In [194]:
df_mercadona[["product_id","reference_price"]].groupby("product_id", as_index=False).max().sort_values(["reference_price"],ascending=False)

,product_id,reference_price
1075,1db0aed2d8e3f844521ff0282c12e9bc,472.973
2461,4613466099f20d600c054508e0aa0ced,470.000
5051,909ea3fc542b59d9c5c8409247269a96,437.500
3871,6e7f839f9d6073364688fe4fc10c34ba,155.000
1030,1c79e0292a686e888e7d1a56692f3a2b,153.334
...,...,...
4290,7aa460df26905b29df81a5e44ce1b258,0.010
969,1af11ec47fe3d0f02068f399c8cd0a4d,0.010
8350,eabc6a3733d0317be1fce4297b012d7a,0.010
1672,2f67d4580a43e70bcc4c255c90292c1b,0.010


In [195]:
df_mercadona.loc[(df_mercadona.product_id=="1db0aed2d8e3f844521ff0282c12e9bc")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]].head(2)

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
48772,Azafrán hebra Hacendado,1.5,405.41,100g,2021-03-06,no
48773,Azafrán hebra Hacendado,1.5,405.41,100g,2021-03-07,no


In [196]:
df_mercadona.loc[(df_mercadona.product_id=="909ea3fc542b59d9c5c8409247269a96")][["name", "price", "reference_price", 
                                               "reference_unit", "insert_date","outlier_reference_price"
                                              ]].head(2)

,name,price,reference_price,reference_unit,insert_date,outlier_reference_price
48116,Azafrán molido Hacendado,1.55,387.5,100g,2021-03-06,no
48117,Azafrán molido Hacendado,1.55,387.5,100g,2021-03-07,no


Por tanto, en este punto quedan corregidos los Outliers en el DataFrame de Mercadona.

In [197]:
#Se borra la columna cocat que se necesita crear de nuevo para trabajar en la columna Name
df_mercadona.drop(["concat"], axis = 1, inplace=True)
df_mercadona.head(3)

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,outlier_price,price_mean,price_lower_limit,price_upper_limit,reference_price_mean,reference_price_lower_limit,reference_price_upper_limit,outlier_reference_price
0,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-06,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,no,4.81,2.45,7.17,1.92,0.77,3.07,no
1,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-07,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,no,4.81,2.45,7.17,1.92,0.77,3.07,no
2,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,Colagen sabor limón Deliplus,Bote,4.8,1.92,100g,2021-03-08,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,no,4.81,2.45,7.17,1.92,0.77,3.07,no


### Columna Name

Aunque esta columna pueda no tener NaN, es necesario unificar los nombres de los productos como se ha hecho en el DF de Carrefour.

In [198]:
#Se crea una nueva columna en el DF que concatene el product_id con el precio de referencia
df_mercadona["concat"] = df_mercadona["product_id"].str.cat(df_mercadona[["reference_price"]].astype(str), sep="-")

#Se ordena el DataFrame por concat y por fecha, para poder obtener el nombre más reciente para cada producto
df_mercadona=df_mercadona.sort_values(['concat', "insert_date"],ascending=False)

#Se crea una nueva columna que tiene para cada producto su descripción más actual.
#En caso de que la descripción más actual esté vacía, utiliza el campo name no vacío más actual.
df_mercadona["name_new"] = df_mercadona.groupby(by = "concat")['name'].transform('first')

#Se vuelve a ordenar el DataFrame por el índice
df_mercadona.sort_index(inplace=True)

print("Hay", len(df_mercadona.loc[df_mercadona.name_new==""].product_id.unique()), "productos sin nombre")
print("Hay", len(df_mercadona.loc[df_mercadona.name_new==""]), "registros sin nombre")

Hay 0 productos sin nombre
Hay 0 registros sin nombre


In [199]:
df_mercadona.isnull().sum()

url                                 0
supermarket                         0
category                            0
name                                0
description                    358618
price                               0
reference_price                     0
reference_unit                      0
insert_date                         0
product_id                          0
Categoria1                          0
Subcategoria1                       0
outlier_price                       0
price_mean                          0
price_lower_limit                   0
price_upper_limit                   0
reference_price_mean                0
reference_price_lower_limit         0
reference_price_upper_limit         0
outlier_reference_price             0
concat                              0
name_new                            0
dtype: int64

Además, aprovechando que en gran medida la columna description tiene los campos rellenos, se va a incluir estos valores en el campo name, para no perderlos ya que la idea es dejar los 3 dataframe con el mismo número de columnas y como en los otros dos se ha borrado, en este también se va a borrar.

In [200]:
#Los valores únicos del campo descripción son
df_mercadona.description.unique()

array(['Bote', 'Caja', 'Tubo', 'Tarro', 'Spray', 'Pack-3', nan, 'Paquete',
       'Sobre', 'Lata', 'Frasco', 'Botella', 'Garrafa', 'Pieza',
       '1/2 Pieza', 'Granel', 'Bandeja', 'Cubo', 'Pack-6', 'Malla',
       'Pack-10', 'Pack-2', 'Vaso', 'Pack-4', 'Tableta', 'Tarrito',
       'Brick', 'Tarrina', 'Manojo', 'Bolsa', '1/4 Pieza', 'Bol',
       'Pastilla', 'Pack-12', 'Saco', 'Pack-8', 'Pack-5', 'Pack-16',
       'Pack-9', 'Botellín', 'Barril', 'Benjamín'], dtype=object)

In [201]:
#Primero se sustituyen los NaN por vacío
df_mercadona["description"] = df_mercadona.description.fillna("")

In [202]:
#Después con un where se incluye el valor de la descripción en el campo name
df_mercadona["name_new"] = np.where(df_mercadona.description!="", 
                                    df_mercadona.name_new + " - " + df_mercadona.description,
                                   df_mercadona.name_new)

In [203]:
#Se comprueba que se ha hecho bien la unión
df_mercadona.loc[df_mercadona.description!=""][["name_new", "description"]]

,name_new,description
0,Colagen sabor limón Deliplus - Bote,Bote
1,Colagen sabor limón Deliplus - Bote,Bote
2,Colagen sabor limón Deliplus - Bote,Bote
3,Colagen sabor limón Deliplus - Bote,Bote
4,Colagen sabor limón Deliplus - Bote,Bote
...,...,...
3852656,Turrón blando de Jijona Antiu Xixona - Tableta,Tableta
3852657,Turrón blando de Jijona Antiu Xixona - Tableta,Tableta
3852658,Turrón blando de Jijona Antiu Xixona - Tableta,Tableta
3852659,Turrón blando de Jijona Antiu Xixona - Tableta,Tableta


### Ultimos pasos
Una vez terminado el trabajo en las columnas, rellenando los campos que faltaban, desglosando las categorías y unificando el campo "name", quedan dos pasos por hacer.
* Eliminar columnas: concat, category, lower_limit, upper_limit, outlier_price  y name que ya se ha trabajado con ellas. Tambien se borra la columna descripción, que ya se ha unido su valor con la columna name_new
* Reordenar las columnas del DataFrame.

In [204]:
#Eliminar columnas sobrantes
df_mercadona.drop(["concat", "name", "description",
            "outlier_price", "price_lower_limit", "price_upper_limit",
            "outlier_reference_price", "reference_price_lower_limit", "reference_price_upper_limit"],
            axis = 1, inplace=True)

df_mercadona.head(2)

,url,supermarket,category,price,reference_price,reference_unit,insert_date,product_id,Categoria1,Subcategoria1,price_mean,reference_price_mean,name_new
0,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,4.8,1.92,100g,2021-03-06,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,4.81,1.92,Colagen sabor limón Deliplus - Bote
1,https://tienda.mercadona.es/product/79439/cola...,Mercadona,fitoterapia_y_parafarmacia_fitoterapia,4.8,1.92,100g,2021-03-07,6dfd858ef08db9863e83c462f25bd5df,Filoterapia y Parafarmacia,Fitoterapia,4.81,1.92,Colagen sabor limón Deliplus - Bote


In [205]:
#Redordenar las columnas
df_mercadona = df_mercadona.reindex(columns=['supermarket',
                                    'name_new',
                                    'price',
                                    'reference_price',
                                    'reference_unit',
                                    'price_mean',
                                    'reference_price_mean',
                                    'category',
                                    'Categoria1',
                                    'Subcategoria1',
                                    'insert_date',
                                    'product_id',
                                    'url'])
df_mercadona.head(3)

,supermarket,name_new,price,reference_price,reference_unit,price_mean,reference_price_mean,category,Categoria1,Subcategoria1,insert_date,product_id,url
0,Mercadona,Colagen sabor limón Deliplus - Bote,4.8,1.92,100g,4.81,1.92,fitoterapia_y_parafarmacia_fitoterapia,Filoterapia y Parafarmacia,Fitoterapia,2021-03-06,6dfd858ef08db9863e83c462f25bd5df,https://tienda.mercadona.es/product/79439/cola...
1,Mercadona,Colagen sabor limón Deliplus - Bote,4.8,1.92,100g,4.81,1.92,fitoterapia_y_parafarmacia_fitoterapia,Filoterapia y Parafarmacia,Fitoterapia,2021-03-07,6dfd858ef08db9863e83c462f25bd5df,https://tienda.mercadona.es/product/79439/cola...
2,Mercadona,Colagen sabor limón Deliplus - Bote,4.8,1.92,100g,4.81,1.92,fitoterapia_y_parafarmacia_fitoterapia,Filoterapia y Parafarmacia,Fitoterapia,2021-03-08,6dfd858ef08db9863e83c462f25bd5df,https://tienda.mercadona.es/product/79439/cola...


Con esto queda "limpio" el DataFrame para poder empezar a trabjar con él y sacar más datos (aparte de la media ya sacada) y poder realizar hipótesis y obtener conclusiones de sus datos.

Después de la limpieza de los datos, el DataFrame queda de la siguiente forma

In [206]:
df_mercadona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3849317 entries, 0 to 3852660
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   supermarket           object        
 1   name_new              object        
 2   price                 float64       
 3   reference_price       float64       
 4   reference_unit        object        
 5   price_mean            float64       
 6   reference_price_mean  float64       
 7   category              object        
 8   Categoria1            object        
 9   Subcategoria1         object        
 10  insert_date           datetime64[ns]
 11  product_id            object        
 12  url                   object        
dtypes: datetime64[ns](1), float64(4), object(8)
memory usage: 411.2+ MB


In [207]:
df_mercadona.isnull().sum()

supermarket             0
name_new                0
price                   0
reference_price         0
reference_unit          0
price_mean              0
reference_price_mean    0
category                0
Categoria1              0
Subcategoria1           0
insert_date             0
product_id              0
url                     0
dtype: int64

In [208]:
print("Quedan", len(df_mercadona.product_id.unique()), "productos únicos y", len(df_mercadona.product_id), "registros en el DataFrame")
print("Se han eliminado",df_mercadona_productos_unicos_inicial-len(df_mercadona.product_id.unique()), "productos únicos y", df_mercadona_registros_inicial-len(df_mercadona.product_id), "registros en el DataFrame")

Quedan 9115 productos únicos y 3849317 registros en el DataFrame
Se han eliminado 7 productos únicos y 6301 registros en el DataFrame


In [209]:
round(df_mercadona.describe(),2)

,price,reference_price,price_mean,reference_price_mean
count,3849317.00,3849317.00,3849317.00,3849317.00
mean,3.33,6.74,3.33,6.74
std,10.32,12.18,9.86,12.15
min,0.08,0.00,0.09,0.01
25%,1.25,1.62,1.26,1.65
50%,2.00,4.00,2.00,4.00
75%,3.75,8.00,3.72,8.00
max,490.05,472.97,466.51,430.99


In [210]:
#El último paso, como se han ido borrando registros es reiniciar el índice.
df_mercadona.reset_index(drop=True, inplace=True)

Con esto queda terminada la limpieza de los tres DataFrame.

# 7. Exportación de los datos

El siguiente paso es exportar estos DataFrame a una base de datos para poder trabajar en la representación de los resultados en Power BI

## 7.1. Preparación de los datos

Para trabajar de forma óptima en Power BI es necesario formar las tablas de dimensiones y las tablas de hecho.

Las tablas de hecho serían los tres DataFrame grandes de Carrefour, Día y Mercadona.

Las tablas de dimensiones serán tablas más pequeñas, formadas a partir de estas tablas de hechos que contengan la información única de los diferentes campos. Las tablas de dimensiones son:
- Tabla para fechas 
- Tabla Categoria
- Tabla para Categoria1
- Tabla para Subcategoria1
- Tabla para reference_unit
- Tabla para supermarket
- Tabla para url

### Tabla fechas

In [211]:
#Se unifican todas las categorías de nive 3 de Carrefour y Dia (meradona no tiene los tres niveles) en un solo DataFrame temporal.
df_fechas=pd.concat([df_carrefour[['insert_date']], 
                            df_dia[['insert_date']],
                      df_mercadona[['insert_date']]],axis = 0)

#Se eliminan las filas repetidas
df_fechas= df_fechas.drop_duplicates()

#Se resetea el índice
df_fechas.reset_index(drop=True, inplace=True)

df_fechas


,insert_date
0,2021-03-06
1,2021-03-07
2,2021-03-08
3,2021-03-09
4,2021-03-10
...,...
630,2022-11-03
631,2021-05-07
632,2021-12-23
633,2022-10-10


### Tabla category
Esta tabla es la que identifica cada categoría completa con el producto, al contar con la columna original de los CSV

In [212]:
#Se unifican todas las categorías de nive 3 de Carrefour y Dia (meradona no tiene los tres niveles) en un solo DataFrame temporal.
df_category=pd.concat([df_carrefour[['category',"Categoria1","Subcategoria1","Subcategoria2", "supermarket"]], 
                            df_dia[['category',"Categoria1","Subcategoria1","Subcategoria2", "supermarket"]],
                      df_mercadona[['category',"Categoria1","Subcategoria1", "supermarket"]]],axis = 0)

#Si hay campos vacíos con NaN se rellenan con vacío
df_category["Subcategoria2"] = df_category.Subcategoria2.fillna("")

#Se eliminan las filas repetidas
df_category= df_category.drop_duplicates()

#Se resetea el índice
df_category.reset_index(drop=True, inplace=True)

#Se crea una columna KEY a partir del índice.
df_category.reset_index(inplace=True)

#Se renombra la columna
df_category.rename(columns={"index":"Categoria_Key"}, inplace=True)

#Se suma 1 a la nueva columna para que no empiece en 0 la Key
df_category["Categoria_Key"] = df_category["Categoria_Key"] + 1

df_category


,Categoria_Key,category,Categoria1,Subcategoria1,Subcategoria2,supermarket
0,1,el_mercado_carniceria_hamburguesas,El Mercado,Carnicería,Hamburguesas,Carrefour
1,2,productos_frescos_carniceria_hamburguesas,Productos Frescos,Carnicería,Hamburguesas,Carrefour
2,3,productos_frescos_sushi_del_dia_carniceria,Productos Frescos,Sushi del día,Carnicería,Carrefour
3,4,el_mercado_carniceria_aves_y_pollo,El Mercado,Carnicería,Aves y Pollo,Carrefour
4,5,el_mercado_carniceria_vacuno,El Mercado,Carnicería,Vacuno,Carrefour
...,...,...,...,...,...,...
1507,1508,charcuteria_y_quesos_pate_y_sobrasada,Charcutería y Quesos,Paté y Sobrasada,,Mercadona
1508,1509,cuidado_del_cabello_peines_y_accesorios,Cuidado del Cabello,Peines y Accesorios,,Mercadona
1509,1510,bodega_cerveza_sin_alcohol,Bodega,Cerveza Sin Alcohol,,Mercadona
1510,1511,maquillaje_pinceles_y_brochas,Maquillaje,Pinceles y Brochas,,Mercadona


### Tabla para Categoria1

In [213]:
#Se unifican todas las categorías principales de los 3 supermercados en un solo DataFrame.
df_categoria1=pd.concat([df_carrefour[["Categoria1", "supermarket"]], 
                         df_dia[["Categoria1", "supermarket"]], 
                         df_mercadona[["Categoria1", "supermarket"]]], axis = 0)

#Se eliminan las filas repetidas
df_categoria1= df_categoria1.drop_duplicates()

#Se resetea el índice
df_categoria1.reset_index(drop=True, inplace=True)

#Se crea una columna KEY a partir del índice.
df_categoria1.reset_index(inplace=True)

#Se renombra la columna
df_categoria1.rename(columns={"index":"Categoria1_Key"}, inplace=True)

#Se suma 1 a la nueva columna para que no empiece en 0 la Key
df_categoria1["Categoria1_Key"] = df_categoria1["Categoria1_Key"] + 1

#Se crea una columna nueva que sea la concatenación de Categoria1 y supermarket para poder introducirlo en la Tabla Categoria
df_categoria1["concat"] = df_categoria1["Categoria1"].str.cat(df_categoria1[["supermarket"]].astype(str), sep="-")

df_categoria1

,Categoria1_Key,Categoria1,supermarket,concat
0,1,El Mercado,Carrefour,El Mercado-Carrefour
1,2,Productos Frescos,Carrefour,Productos Frescos-Carrefour
2,3,La Despensa,Carrefour,La Despensa-Carrefour
3,4,Parafarmacia,Carrefour,Parafarmacia-Carrefour
4,5,Bebidas,Carrefour,Bebidas-Carrefour
5,6,Perfumería e Higiene,Carrefour,Perfumería e Higiene-Carrefour
6,7,Bebé,Carrefour,Bebé-Carrefour
7,8,Limpieza y Hogar,Carrefour,Limpieza y Hogar-Carrefour
8,9,Mascotas,Carrefour,Mascotas-Carrefour
9,10,Droguería y Limpieza,Día,Droguería y Limpieza-Día


### Tabla para Subcategoria1

In [214]:
#Se unifican todas las categorías principales de los 3 supermercados en un solo DataFrame temporal.
df_Subcategoria1=pd.concat([df_carrefour[["Categoria1","Subcategoria1", "supermarket"]], 
                            df_dia[["Categoria1","Subcategoria1", "supermarket"]], 
                            df_mercadona[["Categoria1","Subcategoria1", "supermarket"]]], axis = 0)

#Se eliminan las filas repetidas
df_Subcategoria1= df_Subcategoria1.drop_duplicates()

#Se resetea el índice
df_Subcategoria1.reset_index(drop=True, inplace=True)

#Se crea una columna KEY a partir del índice reseteado.
df_Subcategoria1.reset_index(inplace=True)

#Se renombra la columna
df_Subcategoria1.rename(columns={"index":"Subcategoria1_Key"}, inplace=True)

#Se suma 1 a la nueva columna para que no empiece en 0 la Key
df_Subcategoria1["Subcategoria1_Key"] = df_Subcategoria1["Subcategoria1_Key"] + 1

#Se crea una columna nueva que sea la concatenación de Categoria1 y Subcategoria1 para poder introducirlo en la Tabla Categoria
df_Subcategoria1["concat"] = df_Subcategoria1["Categoria1"].str.cat(df_Subcategoria1[["Subcategoria1", "supermarket"]].astype(str), sep="-")
df_Subcategoria1

,Subcategoria1_Key,Categoria1,Subcategoria1,supermarket,concat
0,1,El Mercado,Carnicería,Carrefour,El Mercado-Carnicería-Carrefour
1,2,Productos Frescos,Carnicería,Carrefour,Productos Frescos-Carnicería-Carrefour
2,3,Productos Frescos,Sushi del día,Carrefour,Productos Frescos-Sushi del día-Carrefour
3,4,Productos Frescos,Charcutería,Carrefour,Productos Frescos-Charcutería-Carrefour
4,5,El Mercado,La Cocina de Carrefour,Carrefour,El Mercado-La Cocina de Carrefour-Carrefour
...,...,...,...,...,...
414,415,Charcutería y Quesos,Paté y Sobrasada,Mercadona,Charcutería y Quesos-Paté y Sobrasada-Mercadona
415,416,Cuidado del Cabello,Peines y Accesorios,Mercadona,Cuidado del Cabello-Peines y Accesorios-Mercadona
416,417,Bodega,Cerveza Sin Alcohol,Mercadona,Bodega-Cerveza Sin Alcohol-Mercadona
417,418,Maquillaje,Pinceles y Brochas,Mercadona,Maquillaje-Pinceles y Brochas-Mercadona


### Actualización Tabla Category

Se actualiza la Tabla Category con las Key de Categoria1 y de Subcategoria1

In [215]:
#Se crea la columna concat creada en la tabla de Subcategoria1 para poder hacer el siguiente merge
df_category["concat"] = df_category["Categoria1"].str.cat(df_category[["supermarket"]].astype(str), sep="-")

#Se hace un df_temporal del df_categoria1 cogiendo solo las columnas categoria y key
df_temporal1 = df_categoria1[["Categoria1_Key", "concat"]]

#Se introduce la Key de Categoria 1 con un merge
df_category= pd.merge(df_category,df_temporal1,how="inner", on=["concat", "concat"])

#Se eliminan las filas repetidas
df_category= df_category.drop_duplicates()

#Se borra la columna concat
df_category.drop(["concat"],axis = 1, inplace=True)

#Se crea la columna concat creada en la tabla de Subcategoria1 para poder hacer el siguiente merge
df_category["concat"] = df_category["Categoria1"].str.cat(df_category[["Subcategoria1", "supermarket"]].astype(str), sep="-")

#Se hace un df_temporal del df_Subcategoria1 cogiendo solo las columnas concat y key
df_temporal2 = df_Subcategoria1[["Subcategoria1_Key", "concat"]]

#Se introduce la Key de Subcategoria 1 con un merge
df_category= pd.merge(df_category,df_temporal2,how="inner", on=["concat", "concat"])

#Se borra la columna concat
df_category.drop(["concat"],axis = 1, inplace=True)

#Se eliminan las filas repetidas
df_category= df_category.drop_duplicates()

#Se reordena el df por la columna de Categoria_key y se reinicia el indice.
df_category=df_category.sort_values(["Categoria_Key"])
df_category.reset_index(drop=True, inplace=True)

#Se cambia el nombre del supermercado por su key
df_category["supermarket"] = np.where(df_category.supermarket=="Carrefour",
                              1, np.where(df_category.supermarket=="Día",
                              2, 3))

df_category

,Categoria_Key,category,Categoria1,Subcategoria1,Subcategoria2,supermarket,Categoria1_Key,Subcategoria1_Key
0,1,el_mercado_carniceria_hamburguesas,El Mercado,Carnicería,Hamburguesas,1,1,1
1,2,productos_frescos_carniceria_hamburguesas,Productos Frescos,Carnicería,Hamburguesas,1,2,2
2,3,productos_frescos_sushi_del_dia_carniceria,Productos Frescos,Sushi del día,Carnicería,1,2,3
3,4,el_mercado_carniceria_aves_y_pollo,El Mercado,Carnicería,Aves y Pollo,1,1,1
4,5,el_mercado_carniceria_vacuno,El Mercado,Carnicería,Vacuno,1,1,1
...,...,...,...,...,...,...,...,...
1507,1508,charcuteria_y_quesos_pate_y_sobrasada,Charcutería y Quesos,Paté y Sobrasada,,3,50,415
1508,1509,cuidado_del_cabello_peines_y_accesorios,Cuidado del Cabello,Peines y Accesorios,,3,39,416
1509,1510,bodega_cerveza_sin_alcohol,Bodega,Cerveza Sin Alcohol,,3,51,417
1510,1511,maquillaje_pinceles_y_brochas,Maquillaje,Pinceles y Brochas,,3,34,418


In [216]:
#Se cambia el nombre del supermercado por su key
df_categoria1["supermarket"] = np.where(df_categoria1.supermarket=="Carrefour",
                              1, np.where(df_categoria1.supermarket=="Día",
                              2, 3))
#Se borra la columna concat
df_categoria1.drop(["concat"],axis = 1, inplace=True)

df_categoria1

,Categoria1_Key,Categoria1,supermarket
0,1,El Mercado,1
1,2,Productos Frescos,1
2,3,La Despensa,1
3,4,Parafarmacia,1
4,5,Bebidas,1
5,6,Perfumería e Higiene,1
6,7,Bebé,1
7,8,Limpieza y Hogar,1
8,9,Mascotas,1
9,10,Droguería y Limpieza,2


In [217]:
#Se cambia el nombre del supermercado por su key
df_Subcategoria1["supermarket"] = np.where(df_Subcategoria1.supermarket=="Carrefour",
                              1, np.where(df_Subcategoria1.supermarket=="Día",
                              2, 3))
#Se borra la columna concat
df_Subcategoria1.drop(["concat"],axis = 1, inplace=True)

df_Subcategoria1

,Subcategoria1_Key,Categoria1,Subcategoria1,supermarket
0,1,El Mercado,Carnicería,1
1,2,Productos Frescos,Carnicería,1
2,3,Productos Frescos,Sushi del día,1
3,4,Productos Frescos,Charcutería,1
4,5,El Mercado,La Cocina de Carrefour,1
...,...,...,...,...
414,415,Charcutería y Quesos,Paté y Sobrasada,3
415,416,Cuidado del Cabello,Peines y Accesorios,3
416,417,Bodega,Cerveza Sin Alcohol,3
417,418,Maquillaje,Pinceles y Brochas,3


### Tabla para reference_unit

In [218]:
#Se unifican todas las columnas de reference_unit de los 3 supermercados en un solo DataFrame.
df_reference_unit=pd.concat([df_carrefour[["reference_unit"]], 
                         df_dia[["reference_unit"]], 
                         df_mercadona[["reference_unit"]],], axis = 0)

#Se eliminan las filas repetidas
df_reference_unit=pd.DataFrame(df_reference_unit["reference_unit"].unique(), columns={"reference_unit"})

#Se crea una columna KEY a partir del índice.
df_reference_unit.reset_index(inplace=True)

#Se renombra la columna
df_reference_unit.rename(columns={"index":"reference_unit_Key"}, inplace=True)

#Se suma 1 a la nueva columna para que no empiece en 0 la Key
df_reference_unit["reference_unit_Key"] = df_reference_unit["reference_unit_Key"] + 1

df_reference_unit

,reference_unit_Key,reference_unit
0,1,kg
1,2,ud
2,3,l
3,4,100g
4,5,100ml
5,6,g
6,7,docena
7,8,lavado
8,9,m


### Tabla para supermarket

In [219]:
#Se unifican todas las columnas supermarket de los 3 supermercados en un solo DataFrame.
df_supermarket=pd.concat([df_carrefour[["supermarket"]], 
                         df_dia[["supermarket"]], 
                         df_mercadona[["supermarket"]],], axis = 0)

#Se eliminan las filas repetidas
df_supermarket=pd.DataFrame(df_supermarket["supermarket"].unique(), columns={"supermarket"})

#Se crea una columna KEY a partir del índice.
df_supermarket.reset_index(inplace=True)

#Se renombra la columna
df_supermarket.rename(columns={"index":"supermarket_Key"}, inplace=True)

#Se suma 1 a la nueva columna para que no empiece en 0 la Key
df_supermarket["supermarket_Key"] = df_supermarket["supermarket_Key"] + 1

df_supermarket

,supermarket_Key,supermarket
0,1,Carrefour
1,2,Día
2,3,Mercadona


### Tabla para url

In [220]:
#Se unifican todas las columnas supermarket de los 3 supermercados en un solo DataFrame.
df_url=pd.concat([df_carrefour[["url"]], 
                         df_dia[["url"]], 
                         df_mercadona[["url"]],], axis = 0)

#Se eliminan las filas repetidas
df_url=pd.DataFrame(df_url["url"].unique(), columns={"url"})

#Se crea una columna KEY a partir del índice.
df_url.reset_index(inplace=True)

#Se renombra la columna
df_url.rename(columns={"index":"url_Key"}, inplace=True)

#Se suma 1 a la nueva columna para que no empiece en 0 la Key
df_url["url_Key"] = df_url["url_Key"] + 1

df_url

,url_Key,url
0,1,https://www.carrefour.es/supermercado/hamburgu...
1,2,https://www.carrefour.es/supermercado/hamburgu...
2,3,https://www.carrefour.es/supermercado/hamburgu...
3,4,https://www.carrefour.es/supermercado/hamburgu...
4,5,https://www.carrefour.es/supermercado/hamburgu...
...,...,...
57542,57543,https://tienda.mercadona.es/product/86031/turr...
57543,57544,https://tienda.mercadona.es/product/67876/turr...
57544,57545,https://tienda.mercadona.es/product/67835/tort...
57545,57546,https://tienda.mercadona.es/product/36050/turr...


## Tablas de Hechos
Una vez creadas las tablas de dimensiones, hay que sustituir sus valores en la tabla de hechos, para dejarla preparada para meterla en la base de datos y trabajar con ella.

Para hacer esto, teniendo en cuenta que el proceso es el mismo para los tres DataFrames, se crea una función que luego se aplica a cada DataFrame

In [221]:
def tablahechos(df):
    
    '''
    Esta función va a recibir un DataFrame y va a trabajar sobre cada una de las columnas relacionadas 
    con las tablas de dimensiones. Agregando la columna con la "key" y borrando la que ya no sirve
    '''
    
    #Columna supermarket
    #Se añade la key de supermarket haciendo un merge con el df_supermarket
    df= pd.merge(df,df_supermarket,how="inner", on=["supermarket", "supermarket"])

    #Columna reference_unit
    #Se añade la key de reference_unit haciendo un merge con el df_reference_unit
    df= pd.merge(df,df_reference_unit,how="inner", on=["reference_unit", "reference_unit"])       
    
    #Columna url
    #Se añade la key de url haciendo un merge con el df_url
    df= pd.merge(df,df_url,how="inner", on=["url", "url"])   
    
    #Columna Category
    #Primero se hace un df_temporal para quedarse solo con las dos columnas que interesan
    df_temporal = df_category[["Categoria_Key", "category"]]
    #Se añade la key de category haciendo un merge con el df_temporal
    df= pd.merge(df,df_temporal,how="inner", on=["category", "category"])   
    
    #Se eliminan posibles filas duplicadas que se hayan creado
    df= df.drop_duplicates()
    
    #Se ordena el dataframe por el índice
    df.sort_index(inplace=True)
    
    #Se resetea el índice
    df.reset_index(drop=True, inplace=True)
    
    #Se eliminan las columnas sobrantes
    #Como el df_mercadona no tiene la columna Subcategoria2 hay que poner un breve condicional
    if "Subcategoria2" in df:
        df.drop(["supermarket", "reference_unit", "Categoria1",
             "Subcategoria1","url", "Subcategoria2", "category"],axis = 1, inplace=True)
    else:
        df.drop(["supermarket", "reference_unit", "Categoria1",
             "Subcategoria1","url","category"],axis = 1, inplace=True)
    
    #Se crea una nueva Key para cada línea de registro en la tabla de hechos a partir del índice.
    df.reset_index(inplace=True)

    #Se renombra la columna
    df.rename(columns={"index":"registro_Key", 'name_new':"name"}, inplace=True)   
    
    #Se suma 1 a la nueva columna para que no empiece en 0 la Key
    df["registro_Key"] = df["registro_Key"] + 1

    #Redordenar las columnas
    df = df.reindex(columns=["registro_Key",
                            'supermarket_Key',
                            'name',
                            'price',
                            'reference_price',
                            'reference_unit_Key',
                            'price_mean',
                            'reference_price_mean',
                            'Categoria_Key',
                            'insert_date',
                            'product_id',
                            'url_Key'])  
    
    return df

Se aplica esta función a los 3 DataFrame para obtener las tablas de hechos de cada uno de ellos

In [222]:
df_carrefour_tablahechos = tablahechos(df_carrefour)

In [223]:
df_dia_tablahechos = tablahechos(df_dia)

In [224]:
df_mercadona_tablahechos = tablahechos(df_mercadona)

Se modifica el título de la columna registro para cada DF para que no se relacionen dentro de PowerBI

In [225]:
df_carrefour_tablahechos.rename(columns={"registro_Key":"registro_Carrefour_Key"}, inplace=True)
df_dia_tablahechos.rename(columns={"registro_Key":"registro_Dia_Key"}, inplace=True)
df_mercadona_tablahechos.rename(columns={"registro_Key":"registro_Mercadona_Key"}, inplace=True)

Se comprueba que no se ha cambiado el tipo de ninguna columna

In [226]:
df_carrefour_tablahechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917059 entries, 0 to 1917058
Data columns (total 12 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   registro_Carrefour_Key  int64         
 1   supermarket_Key         int64         
 2   name                    object        
 3   price                   float64       
 4   reference_price         float64       
 5   reference_unit_Key      int64         
 6   price_mean              float64       
 7   reference_price_mean    float64       
 8   Categoria_Key           int64         
 9   insert_date             datetime64[ns]
 10  product_id              object        
 11  url_Key                 int64         
dtypes: datetime64[ns](1), float64(4), int64(5), object(2)
memory usage: 175.5+ MB


In [227]:
df_dia_tablahechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3697788 entries, 0 to 3697787
Data columns (total 12 columns):
 #   Column                Dtype         
---  ------                -----         
 0   registro_Dia_Key      int64         
 1   supermarket_Key       int64         
 2   name                  object        
 3   price                 object        
 4   reference_price       object        
 5   reference_unit_Key    int64         
 6   price_mean            float64       
 7   reference_price_mean  float64       
 8   Categoria_Key         int64         
 9   insert_date           datetime64[ns]
 10  product_id            object        
 11  url_Key               int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 338.5+ MB


In [228]:
df_mercadona_tablahechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3827932 entries, 0 to 3827931
Data columns (total 12 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   registro_Mercadona_Key  int64         
 1   supermarket_Key         int64         
 2   name                    object        
 3   price                   float64       
 4   reference_price         float64       
 5   reference_unit_Key      int64         
 6   price_mean              float64       
 7   reference_price_mean    float64       
 8   Categoria_Key           int64         
 9   insert_date             datetime64[ns]
 10  product_id              object        
 11  url_Key                 int64         
dtypes: datetime64[ns](1), float64(4), int64(5), object(2)
memory usage: 350.5+ MB


## Tabla de hechos conjunta

In [229]:
#Se unifican las tres tablas de hechos
df_hechoscompleto=pd.concat([df_carrefour_tablahechos, 
                         df_dia_tablahechos, 
                         df_mercadona_tablahechos], axis = 0)

#Se eliminan posibles filas duplicadas que se hayan creado
df_hechoscompleto= df_hechoscompleto.drop_duplicates()

#Se ordena el dataframe por el índice
df_hechoscompleto.sort_index(inplace=True)

#Se resetea el índice
df_hechoscompleto.reset_index(drop=True, inplace=True)

#Se crea una nueva Key para cada línea de registro en la tabla de hechos a partir del índice.
df_hechoscompleto.reset_index(inplace=True)

#Se renombra la columna
df_hechoscompleto.rename(columns={"index":"registro_all_Key"}, inplace=True)   

#Se suma 1 a la nueva columna para que no empiece en 0 la Key
df_hechoscompleto["registro_all_Key"] = df_hechoscompleto["registro_all_Key"] + 1

#Redordenar las columnas
df_hechoscompleto = df_hechoscompleto.reindex(columns=["registro_all_Key",
                                                       'supermarket_Key',
                                                       'name',
                                                       'price',
                                                       'reference_price',
                                                       'reference_unit_Key',
                                                       'price_mean',
                                                       'reference_price_mean',
                                                       'Categoria_Key',
                                                       'insert_date',
                                                       'product_id',
                                                       'url_Key',
                                                       "registro_Carrefour_Key",
                                                       "registro_Dia_Key",
                                                       "registro_Mercadona_Key"])  


#Se rellenan los NaN
df_hechoscompleto = df_hechoscompleto.fillna(0)

df_hechoscompleto

,registro_all_Key,supermarket_Key,name,price,reference_price,reference_unit_Key,price_mean,reference_price_mean,Categoria_Key,insert_date,product_id,url_Key,registro_Carrefour_Key,registro_Dia_Key,registro_Mercadona_Key
0,1,1,Hamburguesa de vacuno raza frisona 240 g,1.95,8.120,1,1.95,8.12,1,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,1,1.0,0.0,0.0
1,2,2,DIA detergente máquina en cápsulas 20 uds,3.79,0.190,8,3.72,0.18,589,2021-03-06,834cbdc30d3f424ab90ea0e78c86a9a5,23453,0.0,1.0,0.0
2,3,3,Colagen sabor limón Deliplus - Bote,4.80,1.920,4,4.81,1.92,1358,2021-03-06,6dfd858ef08db9863e83c462f25bd5df,47525,0.0,0.0,1.0
3,4,1,Hamburguesa de vacuno raza frisona 240 g,1.95,8.120,1,1.95,8.12,1,2021-03-07,fe8eee545a6dc5195dcc29e6e23ca97e,1,2.0,0.0,0.0
4,5,2,DIA detergente máquina en cápsulas 20 uds,3.79,0.190,8,3.72,0.18,589,2021-03-07,834cbdc30d3f424ab90ea0e78c86a9a5,23453,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9442774,9442775,3,"Cerveza sin gluten Free Damm 0,0% sin alcohol ...",0.69,2.091,3,0.69,2.09,1510,2022-12-04,f22405e0d7e67540f31334727278866a,57320,0.0,0.0,3827928.0
9442775,9442776,3,"Cerveza sin gluten Free Damm 0,0% sin alcohol ...",0.69,2.091,3,0.69,2.09,1510,2022-12-05,f22405e0d7e67540f31334727278866a,57320,0.0,0.0,3827929.0
9442776,9442777,3,"Cerveza sin gluten Free Damm 0,0% sin alcohol ...",0.69,2.091,3,0.69,2.09,1510,2022-12-06,f22405e0d7e67540f31334727278866a,57320,0.0,0.0,3827930.0
9442777,9442778,3,"Cerveza sin gluten Free Damm 0,0% sin alcohol ...",0.69,2.091,3,0.69,2.09,1510,2022-12-07,f22405e0d7e67540f31334727278866a,57320,0.0,0.0,3827931.0


In [230]:
df_hechoscompleto.loc[df_hechoscompleto.registro_all_Key==1]

,registro_all_Key,supermarket_Key,name,price,reference_price,reference_unit_Key,price_mean,reference_price_mean,Categoria_Key,insert_date,product_id,url_Key,registro_Carrefour_Key,registro_Dia_Key,registro_Mercadona_Key
0,1,1,Hamburguesa de vacuno raza frisona 240 g,1.95,8.12,1,1.95,8.12,1,2021-03-06,fe8eee545a6dc5195dcc29e6e23ca97e,1,1.0,0.0,0.0


## 7.2. Conexión con la base de datos
Para conectarse a la base de datos se utiliza la función `.connect()`, que en caso de no existir la base de datos, la crea

In [231]:
#Se conecta a la base de datos
conn = sqlite3.connect(r'C:\Users\empca\Documents\14 Bootcamp The Bridge\05 Proyecto EDA\database_eda_supermarket.sqlite')

In [232]:
#Se crea el cursor para poder trabajar con la base de datos
cursor = conn.cursor()

## 7.3. Actualización de los datos

Para facilitar el proceso, aunque no sea lo más óptimo, se va a hacer que cada vez que se actualicen los datos en los CSV y se quieran guardar los DataFrame en la base de datos, primero se borren los datos ya existentes en la base de datos.

Esto tambien se hace así debido a la problemática de los Outliers que se detectan pero que en realidad no lo son, y que con el la sucesiva actualización de la base de datos se van consolidando y por tanto dejan de ser outliers. Si se tuviera que registrar dato a dato podría ser un problema menos óptimo que borrar todo y cargar el resultado de la limpieza.

In [233]:
#Primero se borran las tablas si existen
cursor.execute("DROP TABLE IF EXISTS DimCategoria1")
cursor.execute("DROP TABLE IF EXISTS DimSubcategoria1")
cursor.execute("DROP TABLE IF EXISTS DimCategory")
cursor.execute("DROP TABLE IF EXISTS DimReferenceUnit")
cursor.execute("DROP TABLE IF EXISTS DimSupermarket")
cursor.execute("DROP TABLE IF EXISTS DimURL")
cursor.execute("DROP TABLE IF EXISTS DimDates")
cursor.execute("DROP TABLE IF EXISTS FactComplete")

In [234]:
#Después se cargan las tablas de dimensiones
df_categoria1.to_sql("DimCategoria1", con= conn, index=False)
df_Subcategoria1.to_sql("DimSubcategoria1", con= conn, index=False)
df_category.to_sql("DimCategory", con= conn, index=False)
df_reference_unit.to_sql("DimReferenceUnit", con= conn, index=False)
df_supermarket.to_sql("DimSupermarket", con= conn, index=False)
df_url.to_sql("DimURL", con= conn, index=False)
df_fechas.to_sql("DimDates", con= conn, index=False)

635

In [235]:
#Se carga la tabla de hechos Completa
df_hechoscompleto.to_sql("FactComplete", con= conn, index=False)

9442779

In [236]:
#Se comprueba que se han guardado
pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', conn)

,name
0,DimCategoria1
1,DimSubcategoria1
2,DimCategory
3,DimReferenceUnit
4,DimSupermarket
5,DimURL
6,DimDates
7,FactComplete


In [237]:
#Esta celda des de comprobación de cuanto tarda en hacer todo el proceso completo.
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Hora de inicio =", inicio)
print("Hora de fin =", current_time)    

Hora de inicio = 18:48:24
Hora de fin = 18:59:40


A partir de aquí, se trabajar en Power BI para la visualización de resultados.

Los datos se pueden en el siguiente [enlace](https://app.powerbi.com/reportEmbed?reportId=7dc237a7-855d-4e07-bc74-582495889db5&autoAuth=true&ctid=24a632fe-9754-48c6-b0a6-be004693446d)